In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

# 식당 데이터셋 전처리

## 기본 전처리

In [ ]:
rest_1 = pd.read_csv('/content/drive/MyDrive/제주숙박주변식당/HW_JJ_LDGS_CFR_RSTRNT_PREFEER_INFO_202303.csv')
rest_2 = pd.read_csv('/content/drive/MyDrive/제주숙박주변식당/HW_JJ_LDGS_CFR_RSTRNT_PREFEER_INFO_202306.csv')
rest_3 = pd.read_csv('/content/drive/MyDrive/제주숙박주변식당/HW_JJ_LDGS_CFR_RSTRNT_PREFEER_INFO_202309.csv')

hotel_location = pd.read_csv('/content/drive/MyDrive/Accommodation_Facility_Information_df(위경도추가) (1).csv')

In [ ]:
# 데이터 병합 및 컬럼 정리
jeju_restaurant_df = pd.concat([rest_1, rest_2, rest_3], axis=0)

jeju_restaurant_df.rename(columns={'BASE_YM':'기준년월','LDGS_NM':'숙박업명','LDGS_ADDR':'숙박업주소','LCLAS_NM':'대분류명','MLSFC_NM':'중분류명',
                     'RSTRNT_NM':'식당명','CTY_NM':'도시명','ADSTRD_NM':'행정동명','RSTRNT_ADDR':'식당주소','RSTRNT_LA':'식당위도',
                     'RSTRNT_LO':'식당경도','SCCNT_SM_VALUE':'검색량합계값','JJINHBT_SALES_PRICE_RATE':'제주도민매출금액비율', 'OTSD_SALES_PRICE_RATE':'외지인매출금액비율','ALL_SALES_PRICE_RATE':'전체매출금액비율'},inplace=True)
jeju_restaurant_df = jeju_restaurant_df.drop(['기준년월','중분류명','행정동명','제주도민매출금액비율','외지인매출금액비율','전체매출금액비율'], axis=1)

In [ ]:
# 숙박업 위경도 컬럼 병합
jeju_restaurant_df = pd.merge(hotel_location, jeju_restaurant_df, on='숙박업명', how='inner')

In [ ]:
# 불필요 컬럼 삭제 및 컬럼명 변경
jeju_restaurant_df.drop(['시도명', '구군명', '주변숙박업목록내용', '1분기평균가격', '2분기평균가격', '객실수',
       '주차장존재여부', '식당존재여부', '바여부', '카페보유여부', '피트니스센터여부', '야외수영장여부', '스파여부',
       '사우나여부', '연회장여부', '비즈니스센터여부', '해변여부'],axis=1,inplace=True)
jeju_restaurant_df.rename(columns={'위도':'숙박위도','경도':'숙박경도'}, inplace=True)
jeju_restaurant_df = jeju_restaurant_df[['숙박업명','도시명','숙박업주소','숙박위도','숙박경도','대분류명','식당명','식당주소','식당위도','식당경도','검색량합계값']]

In [ ]:
jeju_restaurant_df.head()

,숙박업명,도시명,숙박업주소,숙박위도,숙박경도,대분류명,식당명,식당주소,식당위도,식당경도,검색량합계값
0,히든 클리프 호텔 앤 네이처,서귀포시,제주특별자치도 서귀포시 상예동 625,33.254885,126.402341,한식,덤장중문점,제주 서귀포시 천제연로 17,33.257709,126.408190,232
1,히든 클리프 호텔 앤 네이처,서귀포시,제주특별자치도 서귀포시 상예동 625,33.254885,126.402341,한식,중문해물라면오빠네본점,제주 서귀포시 일주서로 962,33.258310,126.407954,321
2,히든 클리프 호텔 앤 네이처,서귀포시,제주특별자치도 서귀포시 상예동 625,33.254885,126.402341,고기,신우성흑돼지,제주 서귀포시 중문관광로72번길 29-9,33.249636,126.411416,116
3,히든 클리프 호텔 앤 네이처,서귀포시,제주특별자치도 서귀포시 상예동 625,33.254885,126.402341,한식,형제도식당,제주 서귀포시 일주서로 915,33.257860,126.412995,454
4,히든 클리프 호텔 앤 네이처,서귀포시,제주특별자치도 서귀포시 상예동 625,33.254885,126.402341,고기,숙성도중문점,제주 서귀포시 일주서로 966,33.258585,126.407637,2715


In [ ]:
print(f'제주도 식당 총 갯수: {len(jeju_restaurant_df["식당명"].unique())}개')

제주도 식당 총 갯수: 972개


In [ ]:
# 숙박업명과 식당명으로 그룹화 & 숙박업명 오름차순/검색량합계값 내림차순
def customize(group):
    aggregated_row = {
        '숙박업명': group['숙박업명'].iloc[0],
        '도시명': group['도시명'].iloc[0],
        '숙박업주소': group['숙박업주소'].iloc[0],
        '숙박위도': group['숙박위도'].iloc[0],
        '숙박경도': group['숙박경도'].iloc[0],
        '대분류명': group['대분류명'].iloc[0],
        '식당명': group['식당명'].iloc[0],
        '식당주소': group['식당주소'].iloc[0],
        '식당위도': group['식당위도'].iloc[0],
        '식당경도': group['식당경도'].iloc[0],
        '검색량합계값': group['검색량합계값'].sum()
    }
    return pd.Series(aggregated_row, index=aggregated_row.keys())

jeju_restaurant_df = jeju_restaurant_df.groupby(['숙박업명', '식당명']).apply(customize).reset_index(drop=True)
jeju_restaurant_df = jeju_restaurant_df.sort_values(by=['숙박업명', '검색량합계값'], ascending=[True, False])

In [ ]:
# 각 숙박업별로 몇개의 추천 식당을 가지는가?
jeju_restaurant_df['숙박업명'].value_counts()

숙박업명
골든데이지호텔서귀포오션              118
제주 노블피아 호텔                116
호텔 케니 서귀포                 113
오션팰리스                     109
헤이, 서귀포                   108
신신호텔 서귀포점                 106
카사로마 호텔                   100
신신호텔 천지연                   91
제주 센트럴 시티                  90
신라스테이제주                    89
아시아호텔 제주                   88
호텔 앨리스 앤 트렁크               87
호텔 위드스테이 제주                87
호텔위드제주                     87
위드시티호텔                     87
호텔 레오 제주                   86
엠버호텔                       85
어반스테이 제주연동                 84
롯데시티호텔 제주                  79
롯데호텔 제주                    79
오션 스위츠 제주 호텔               78
호텔 스카이파크 제주 1호점            77
휘슬락                        77
엠버시티호텔(구.아이비호텔)            77
베스트웨스턴 제주호텔                77
제주 리젠트마린 더 블루              77
그랜드하얏트 제주                  77
노형호텔                       76
호텔더원                       76
신신호텔 제주공항                  72
글로스터 제주                    71
어반아일랜드호텔                   70
아이진호텔                      63
호텔 시리

> 식당목록을 50개 미만으로 가지는 숙박업소에서 추천해주는 것이 과연 맞을까?

## 위도/경도 추가

In [ ]:
# 위도와 경도 기반 거리 구하는 라이브러리
!pip install haversine

from haversine import haversine

# 거리를 계산하여 새로운 컬럼 형성
jeju_restaurant_df['숙박_식당 거리(km)'] = jeju_restaurant_df.apply(
    lambda row: haversine((row['숙박위도'], row['숙박경도']), (row['식당위도'], row['식당경도'])),
    axis=1  # 각 행에 적용하기 위해 axis=1 지정
)

In [ ]:
jeju_restaurant_df['숙박_식당 거리(km)'] = jeju_restaurant_df['숙박_식당 거리(km)'].round(3)

In [ ]:
jeju_restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4310 entries, 62 to 4292
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   숙박업명          4310 non-null   object 
 1   도시명           4310 non-null   object 
 2   숙박업주소         4310 non-null   object 
 3   숙박위도          4310 non-null   float64
 4   숙박경도          4310 non-null   float64
 5   대분류명          4310 non-null   object 
 6   식당명           4310 non-null   object 
 7   식당주소          4310 non-null   object 
 8   식당위도          4310 non-null   float64
 9   식당경도          4310 non-null   float64
 10  검색량합계값        4310 non-null   int64  
 11  숙박_식당 거리(km)  4310 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 437.7+ KB


* 치완님 데이터셋에서 제외되는 호텔 5곳('WE 호텔 제주', '베스트웨스턴 제주', '엠버리조트', '호텔 토스카나', '켄싱턴리조트 제주중문')

# 치완님 데이터셋

In [ ]:
hotel_df = pd.read_csv('/content/drive/MyDrive/공모전_군집분석_최종완성끝끝끝!!.csv')
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   숙박업명          96 non-null     object 
 1   구군명           96 non-null     object 
 2   주차장존재여부       96 non-null     object 
 3   식당존재여부        96 non-null     object 
 4   바여부           96 non-null     object 
 5   카페보유여부        96 non-null     object 
 6   피트니스센터여부      96 non-null     object 
 7   야외수영장여부       96 non-null     object 
 8   스파여부          96 non-null     object 
 9   사우나여부         96 non-null     object 
 10  연회장여부         96 non-null     object 
 11  비즈니스센터여부      96 non-null     object 
 12  해변여부          96 non-null     object 
 13  위도            96 non-null     float64
 14  경도            96 non-null     float64
 15  주변숙박업소수       96 non-null     int64  
 16  관광지_5km_내_개수  96 non-null     int64  
 17  객실수_구간        96 non-null     object 
 18  평균가격_구간       96 non-null     ob

In [ ]:
hotel_df.head(5)

,숙박업명,구군명,주차장존재여부,식당존재여부,바여부,카페보유여부,피트니스센터여부,야외수영장여부,스파여부,사우나여부,연회장여부,비즈니스센터여부,해변여부,위도,경도,주변숙박업소수,관광지_5km_내_개수,객실수_구간,평균가격_구간,공항과의거리_구간,군집,편의시설,주변 관광지,가격대,공항과의 거리
0,히든 클리프 호텔 앤 네이처,서귀포시,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,33.254885,126.402341,4,30,중간 규모,고가,원거리,4,"주차장, 식당, 바, 카페, 피트니스센터, 야외수영장, 스파, 사우나 비율이 매우 높음",평균 관광지 수 (5km 내): 21.5개,가격대: 높음,공항과의 거리: 원거리
1,스위트 호텔 제주,서귀포시,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,33.249332,126.408394,5,19,소규모,고가,원거리,4,"주차장, 식당, 바, 카페, 피트니스센터, 야외수영장, 스파, 사우나 비율이 매우 높음",평균 관광지 수 (5km 내): 21.5개,가격대: 높음,공항과의 거리: 원거리
2,제주 신라 호텔,서귀포시,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,33.247163,126.407955,5,18,대규모,초고가,원거리,4,"주차장, 식당, 바, 카페, 피트니스센터, 야외수영장, 스파, 사우나 비율이 매우 높음",평균 관광지 수 (5km 내): 21.5개,가격대: 높음,공항과의 거리: 원거리
3,롯데호텔 제주,제주시,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,33.248447,126.410606,22,19,대규모,초고가,원거리,4,"주차장, 식당, 바, 카페, 피트니스센터, 야외수영장, 스파, 사우나 비율이 매우 높음",평균 관광지 수 (5km 내): 21.5개,가격대: 높음,공항과의 거리: 원거리
4,WE 호텔 제주,서귀포시,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,N,33.286140,126.444209,0,10,중간 규모,고가,원거리,4,"주차장, 식당, 바, 카페, 피트니스센터, 야외수영장, 스파, 사우나 비율이 매우 높음",평균 관광지 수 (5km 내): 21.5개,가격대: 높음,공항과의 거리: 원거리


In [ ]:
map_df = pd.read_csv('/content/drive/MyDrive/맵자료정리.csv')
map_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관광지명    627 non-null    object 
 1   관광지분류   627 non-null    object 
 2   주소      627 non-null    object 
 3   운영상세사항  576 non-null    object 
 4   관광지설명   607 non-null    object 
 5   경도      627 non-null    float64
 6   위도      627 non-null    float64
 7   분류      627 non-null    object 
dtypes: float64(2), object(6)
memory usage: 39.3+ KB


In [ ]:
map_df.head()

,관광지명,관광지분류,주소,운영상세사항,관광지설명,경도,위도,분류
0,금호제주리조트 아쿠아나,레저/테마,제주특별자치도 서귀포시 남원읍 남원리 2390,매월1번째월요일 /,바다와 한라산 전망을 동시에 누리는 Seaside Resort / 11000원(온천...,126.701433,33.272643,마을 관광자원
1,큰엉해안경승지,관광지,제주특별자치도 서귀포시 남원읍 남원리 2379-3,연중무휴 / (평일+주말+공휴일)11:00~20:00,때묻지 않은 기암절벽이 아름다운 해양경승지 / 무료,126.703709,33.273343,마을 관광자원
2,코코몽 에코파크,레저/테마,제주특별자치도 서귀포시 남원읍 남원리 2380,화요일 / (평일+주말+공휴일)10:00~18:00,아이들의 오감을 자극하는 친환경 프리미엄 테마파크 / 25000원(24개월이상13세...,126.705851,33.274558,마을 관광자원
3,마흐니 숲길,관광지,제주특별자치도 서귀포시 남원읍 수망리 산203,연중무휴 /,사색을 즐기며 걷기 좋은 숲길 / 무료,126.655193,33.379447,마을 관광자원
4,물영아리오름,관광지,제주특별자치도 서귀포시 남원읍 수망리 산 189,연중무휴 /,신령스러운 산이라는 뜻의 '영아리'와 분화구에 물이 고인 습지를 품고 있기에 '물'...,126.693520,33.370809,마을 관광자원


In [ ]:
# jeju_restaurant_df에서 hotel_df에 없는 숙박업명 필터링 --> 다 있는 것으로 확인
filtered_jeju_restaurant_df = jeju_restaurant_df[jeju_restaurant_df['숙박업명'].isin(hotel_df['숙박업명'])]
filtered_jeju_restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4310 entries, 62 to 4292
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   숙박업명          4310 non-null   object 
 1   도시명           4310 non-null   object 
 2   숙박업주소         4310 non-null   object 
 3   숙박위도          4310 non-null   float64
 4   숙박경도          4310 non-null   float64
 5   대분류명          4310 non-null   object 
 6   식당명           4310 non-null   object 
 7   식당주소          4310 non-null   object 
 8   식당위도          4310 non-null   float64
 9   식당경도          4310 non-null   float64
 10  검색량합계값        4310 non-null   int64  
 11  숙박_식당 거리(km)  4310 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 437.7+ KB


In [ ]:
# jeju_restaurant_df.to_csv('/content/drive/My Drive/제주숙박식당정보_최종본.csv', encoding='utf-8-sig', index=False)

# 리뷰 데이터셋 전처리

### 기본 전처리

In [19]:
review_df = pd.read_csv('/content/drive/MyDrive/네이버_크롤링_식당/naver_review_final.csv')
review_df1 = pd.read_csv('/content/drive/MyDrive/네이버_크롤링_식당/naver_review_final_2.csv')
review_df2 = pd.read_csv('/content/drive/MyDrive/네이버_크롤링_식당/naver_review_final_3.csv')
review_df3 = pd.read_csv('/content/drive/MyDrive/네이버_크롤링_식당/naver_review_final4.csv')

In [20]:
review_df = pd.concat([review_df, review_df1, review_df2, review_df3], axis=0)
review_df.rename(columns={'restaurant_name':'식당명', 'restaurant_info':'식당정보','nickname':'닉네임','content':'리뷰'}, inplace=True)

In [21]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101053 entries, 0 to 3356
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   식당명     101053 non-null  object
 1   식당정보    101053 non-null  object
 2   닉네임     101053 non-null  object
 3   리뷰      98220 non-null   object
dtypes: object(4)
memory usage: 3.9+ MB


In [22]:
review_df.sample(3)

,식당명,식당정보,닉네임,리뷰
25658,성산물고기자리,별점\n4.64방문자리뷰 312블로그리뷰 106,kwd****,알탕맛집
29251,일송회수산,별점\n4.49방문자리뷰 721블로그리뷰 545\n서귀포횟집추천 싱싱한 활어,마키C,자연산활어라 그런지 진짜 신선하고 양이 많았어요!\n현지인맛집 찐입니다
32775,성산을본고등어쌈밥 제주성산일출봉점,"방문자리뷰 889블로그리뷰 1,657\n농림축산식품부 제공\n안심식당",shi****,고등어쌈밥과 고등어구이를 오랜만에 먹었는데 밑반찬까지 너무 맛있게 먹었어요!! 묵은...


In [29]:
print(f'제주도 리뷰 식당 총 갯수: {len(review_df["식당명"].unique())}개')

제주도 리뷰 식당 총 갯수: 932개


> 식당별 리뷰 총 갯수 몇 개 이하면 자를지?

In [35]:
# 식당별 리뷰 갯수
review_df['식당명'].value_counts().head(30)

식당명
중앙식당            560
수선화식당           540
옹포바다횟집          520
칠돈가 서귀포점        510
강실장회포차          456
예소담 도두점         322
칠돈가 본점          310
흑돈쭐 제주성산일출봉점    310
만선식당            310
삼대국수회관협재점       310
애월흑돼지 하귀 흑생     310
어도식당            305
삼무국수 2호점        300
제주할망밥상 애월점      300
신우성흑돼지          300
하르방밀면 노형점       300
돌카롱 제주공항점       300
아라파파            300
덕승식당            300
60빈스            300
피제리아3657        300
난드르바당           300
카페1950          300
오늘의바다           300
제주할망뚝배기         300
제주미담            300
백기해녀의집          300
백한철꽈배기&식빵       300
산지해장국 성산점       300
사계전복 산방산본점      300
Name: count, dtype: int64

In [36]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101053 entries, 0 to 3356
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   식당명     101053 non-null  object
 1   식당정보    101053 non-null  object
 2   닉네임     101053 non-null  object
 3   리뷰      98220 non-null   object
dtypes: object(4)
memory usage: 3.9+ MB


In [37]:
# 리뷰 없는 행 삭제
review_df = review_df.dropna(subset='리뷰')
review_df = review_df.reset_index(drop=True)

In [38]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98220 entries, 0 to 98219
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   식당명     98220 non-null  object
 1   식당정보    98220 non-null  object
 2   닉네임     98220 non-null  object
 3   리뷰      98220 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


In [39]:
# '평점' 컬럼 생성
review_df['평점'] = review_df['식당정보'].str.extract(r'별점\n(\d+\.\d+)')
review_df.head()

,식당명,식당정보,닉네임,리뷰,평점
0,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,Sorigin,"닭반마리칼국수와 1인만두입니다. 밥, 면 리필 가능한 식당입니다! 저녁 8시에 갔는...",4.33
1,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,쩨쩨Ol,제주도에 사는 친구의 찐 추천을 받아 방문했는데 정말 맛있었어요!!! 사람수대로 주...,4.33
2,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,dry91,닭칼국수시켰는데 닭다리 하나씩 들어있어요. 김치도 매일 생지를 담그시고 묵은지도 있...,4.33
3,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,whi****,반마리칼국수 맛있네요! 국물도진득하게~~면도쫄깃하고 만두가육즙가득한게진짜맛있오요!!,4.33
4,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,맛집카페술집여행소통블로거,12시 이전에 가니 기다리지 않아서 다행 두번째 만강촌 닭칼은 쫌 짭고 닭녹두죽은 ...,4.33


In [40]:
review_df['평점'] = review_df['평점'].astype('float').round(2)

In [41]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98220 entries, 0 to 98219
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   식당명     98220 non-null  object 
 1   식당정보    98220 non-null  object 
 2   닉네임     98220 non-null  object 
 3   리뷰      98220 non-null  object 
 4   평점      67155 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.7+ MB


In [42]:
review_df['평점'].describe()

count    67155.000000
mean         4.482500
std          0.190468
min          3.670000
25%          4.350000
50%          4.470000
75%          4.600000
max          4.960000
Name: 평점, dtype: float64

In [43]:
review_df['평점'].isnull().sum() # 3.67점 이상 평점만 존재 -> 평점이 낮고 높고로 평가 불가 (부가정보로만 활용)

31065

In [44]:
review_df['리뷰'].duplicated().sum() # 리뷰 중복은 어쩔 수 없음 ex) 굿, 좋아요 이런 리뷰가 많음

13992

### 리뷰 포함 특수문자 및 숫자 제거

In [ ]:
import re

def remove_non_korean(text):
    """
    한글과 공백을 제외한 모든 문자를 제거하는 함수
    """
    return re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', text)

review_df['리뷰'] = review_df['리뷰'].apply(remove_non_korean)

In [ ]:
review_df['리뷰'] = review_df['리뷰'].astype('str')

In [ ]:
# 한글 문자열 제외한 나머지 문자 공백 처리
def extract_word(text):
    hangul = re.compile('[^가-힣]')
    result = hangul.sub(' ', text)
    return result

In [ ]:
review_df['리뷰'] = review_df['리뷰'].apply(lambda x:extract_word(x)) #적용

In [ ]:
review_df.head()

,식당명,식당정보,닉네임,리뷰,평점
0,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,Sorigin,닭반마리칼국수와 인만두입니다 밥 면 리필 가능한 식당입니다 저녁 시에 갔는데 꽤 많...,4.33
1,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,쩨쩨Ol,제주도에 사는 친구의 찐 추천을 받아 방문했는데 정말 맛있었어요 사람수대로 주문하면...,4.33
2,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,dry91,닭칼국수시켰는데 닭다리 하나씩 들어있어요 김치도 매일 생지를 담그시고 묵은지도 있고...,4.33
3,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,whi****,반마리칼국수 맛있네요 국물도진득하게면도쫄깃하고 만두가육즙가득한게진짜맛있오요,4.33
4,만강촌옛날칼국수,별점\n4.33방문자리뷰 991블로그리뷰 318,맛집카페술집여행소통블로거,시 이전에 가니 기다리지 않아서 다행 두번째 만강촌 닭칼은 쫌 짭고 닭녹두죽은 싱거...,4.33


In [ ]:
# review_df.to_csv('/content/drive/My Drive/JEJU_1ST.csv', encoding='utf-8-sig')

### PyKoSpacing_ 띄어쓰기 딥러닝 모델

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-fiap0vgd
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-fiap0vgd
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 39.5 MB/s eta 0:00:00
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286919 sha256=7af62b4d456a199e7535f27ab5eca9566af927eab87bfc06d8a05391eef

In [ ]:
sent = '여긴 예약을 꼭 해야할듯  좀 바빠지면서 음식이 좀 늦게나와서 아쉬움이 남을뻔했디만 그걸 다 카바하는 직원들 클라스'

In [ ]:
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

여긴예약을꼭해야할듯좀바빠지면서음식이좀늦게나와서아쉬움이남을뻔했디만그걸다카바하는직원들클라스


In [ ]:
# Spacing() 활용한 한국어 띄어쓰기 적용

from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent)

print(sent)
print(kospacing_sent)

여긴 예약을 꼭 해야할듯  좀 바빠지면서 음식이 좀 늦게나와서 아쉬움이 남을뻔했디만 그걸 다 카바하는 직원들 클라스
여긴 예약을 꼭 해야 할 듯 좀 바빠지면서 음식이 좀 늦게 나와서 아쉬움이 남을 뻔했디만 그걸 다카바하는 직원들 클라스


In [ ]:
spacing = Spacing()
print("Before Fixing : ",review_df['리뷰'][7992])
print("After Fixing : ", spacing(review_df['리뷰'][7992]))

Before Fixing :  공항 근처 흑돼지집 찾아서 갔는데 고기 육즙이 미쳤어요 직접 구워주시고 초반에 설명과 함께 소스까지 찍어 주셔서 좋았어요 감사합니당
After Fixing :  공항 근처 흑돼지 집 찾아서 갔는데 고기 육즙이 미쳤어요 직접 구워주시고 초반에 설명과 함께 소스까지 찍어 주셔서 좋았어요 감사합니당


In [ ]:
from tqdm import tqdm

# Spacing 객체 생성
spacing = Spacing()

# tqdm을 이용하여 review_df['content']의 모든 텍스트에 Spacing 적용
for idx, text in tqdm(enumerate(review_df['리뷰']), total=len(review_df)):
    review_df.loc[idx, '리뷰'] = spacing(text)

100%|██████████| 98220/98220 [1:00:56<00:00, 26.86it/s]


In [ ]:
# review_df.to_csv('/content/drive/My Drive/제주도식당리뷰_띄어쓰기_1.csv', encoding='utf-8-sig')

### 형태소 분석

In [ ]:
review_df = pd.read_csv('/content/drive/MyDrive/제주도식당리뷰_띄어쓰기_1.csv')
review_df = review_df.drop(['Unnamed: 0'], axis=1)
review_df.rename(columns={'restaurant_name':'식당명', 'restaurant_info':'식당정보','nickname':'닉네임','content':'리뷰','rating':'평점','review_count':'방문자/블로그리뷰'}, inplace=True)
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98220 entries, 0 to 98219
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   식당명     98220 non-null  object 
 1   식당정보    98220 non-null  object 
 2   닉네임     98220 non-null  object 
 3   리뷰      96882 non-null  object 
 4   평점      67155 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.7+ MB


In [ ]:
len(review_df['식당명'].unique())

931

In [ ]:
# 방문자/블로그 리뷰 컬럼 생성

import re
# 방문자 리뷰와 블로그 리뷰 추출하여 review_count 컬럼 생성하는 함수
def extract_review_count(info):
    match = re.search(r'방문자리뷰 ([\d,]+)블로그리뷰 (\d+,?\d*)', info)
    if match:
        visitor_reviews = int(match.group(1).replace(',', ''))  # 방문자 리뷰 수 (쉼표 제거)
        blog_reviews = int(match.group(2).replace(',', ''))  # 블로그 리뷰 수 (쉼표 제거)
        return visitor_reviews + blog_reviews
    else:
        return None

# apply 함수를 사용하여 review_count 컬럼 생성
review_df['방문자/블로그리뷰'] = review_df['식당정보'].apply(extract_review_count)
review_df.sample(3)

,식당명,식당정보,닉네임,리뷰,평점,방문자/블로그리뷰
64853,성산을본고등어쌈밥 제주성산일출봉점,"방문자리뷰 889블로그리뷰 1,657\n농림축산식품부 제공\n안심식당",희릐하이힣,너무 맛있어요 제주 여행 갔다가 고등어 맛집 찾아봤는데 여기가 떠서 왔어요 친구도 ...,NaN,2546.0
59747,독개물항 제주한림 협재 본점,"별점\n4.34방문자리뷰 2,229블로그리뷰 7,747\n조림과 뚝배기로 유명한 한림맛집",영심이7,모든 음식들 맛있어요,4.34,9976.0
4984,애월 해성도뚜리,"별점\n4.45방문자리뷰 1,711블로그리뷰 4,452\n맛있는제주9호점 토마토짬뽕...",gah****,너무 너 무너무 맛있어요 뷰만 맛집이 아니라 음식도 맛집,4.45,6163.0


In [ ]:
# 방문자 리뷰와 블로그 리뷰에서 특성 추출하여 review_feature 컬럼 생성하는 함수
def extract_review_feature(info):
    match = re.search(r'방문자리뷰 (\d+(?:,\d+)?)블로그리뷰 (\d+(?:,\d+)?)\s*([\w\s,]*)', info)
    if match:
        visitor_reviews = match.group(1)
        blog_reviews = match.group(2)
        features = match.group(3).strip().replace('\n', ',')  # "\n"을 쉼표로 변경
        return features
    else:
        return None

# apply 함수를 사용하여 review_feature 컬럼 생성
review_df['식당정보'] = review_df['식당정보'].apply(extract_review_feature)
review_df.tail(3)

,식당명,식당정보,닉네임,리뷰,평점,방문자/블로그리뷰
98217,해오반 함덕점,"농림축산식품부 제공,안심식당",악당4659,아이들 포함 명이 가서 세트시켰는데 너무 잘 먹었습니다 반찬들도 맛있고 다양하고 좋았네요,NaN,18087.0
98218,해오반 함덕점,"농림축산식품부 제공,안심식당",kab****,와 진짜 끝내줘요 두 말 하면 잔 소리 임 함덕 맛집이라고 친구가 소개해줘서 갔는데...,NaN,18087.0
98219,해오반 함덕점,"농림축산식품부 제공,안심식당",rud****,딱새우랑 이것저것 시켜서 먹었는데 다 맛있게 먹었습니다 제주도 함 덕에서 해산물 최...,NaN,18087.0


In [ ]:
# 리뷰 없는 행 삭제
review_df = review_df.dropna(subset=['리뷰'])
review_df = review_df.reset_index(drop=True)

In [ ]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96882 entries, 0 to 96881
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   식당명        96882 non-null  object 
 1   식당정보       96831 non-null  object 
 2   닉네임        96882 non-null  object 
 3   리뷰         96882 non-null  object 
 4   평점         66096 non-null  float64
 5   방문자/블로그리뷰  96831 non-null  float64
dtypes: float64(2), object(4)
memory usage: 4.4+ MB


In [ ]:
len(review_df['식당명'].unique())

931

In [ ]:
!pip install konlpy
from konlpy.tag import Okt
from collections import Counter

# Okt 형태소 분석기 생성
okt = Okt()

# 한글불용어.txt 파일 불러오기 (따로 만들어서 적용함)
with open('/content/drive/MyDrive/한글불용어.txt', 'r', encoding='utf-8') as f:
  stopwords = f.readlines()
stopwords = [word.strip() for word in stopwords]  # 불용어 리스트 생성


# 조사 제외 후 불용어 처리
def remove_stopwords_josa_number_english(text):
     tokens = okt.pos(text)  # 형태소 분석 및 품사 태깅
     filtered_tokens = [word for word, pos in tokens if pos not in ['Josa'] and word not in stopwords]
     filtered_text = ' '.join(filtered_tokens)
     return filtered_text

review_df['review'] = review_df['리뷰'].apply(remove_stopwords_josa_number_english)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 49.2 MB/s eta 0:00:00


In [ ]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96882 entries, 0 to 96881
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   식당명        96882 non-null  object 
 1   식당정보       96831 non-null  object 
 2   닉네임        96882 non-null  object 
 3   리뷰         96882 non-null  object 
 4   평점         66096 non-null  float64
 5   방문자/블로그리뷰  96831 non-null  float64
 6   review     96882 non-null  object 
dtypes: float64(2), object(5)
memory usage: 5.2+ MB


In [ ]:
review_df.sample(10)

,식당명,식당정보,닉네임,리뷰,평점,방문자/블로그리뷰,review
69342,마농삼계탕,,풍돈,맛있음 그것 하나로 충분,4.35,174.0,맛있음 그것 하나로 충분
80473,소낭 비프 하우스 제원,"농림축산식품부 제공,안심식당",jeo****,고기 부드럽고 맛있어요 매장 분위기도 좋아요 데이트 장소로 딱이에요,NaN,2097.0,고기 부드럽고 맛있어요 분위기 데이트 장소 딱이에요
64639,함덕다퍼주는횟집,"농림축산식품부 제공,안심식당",쌔미9,함덕가면 여기 꼭 가여,4.70,2325.0,함덕 가여
8733,제주어멍 통갈치 제주공항 본점,,오르뎅,바로 바다가 보여서 뷰가 멋졌어요 갈치조림 시키면 고등어 구이랑 돔 베고 기도 나와...,4.37,3609.0,바다 뷰 멋졌어요 갈치 조림 고등어 구이 돔 베고 가지 맛 볼 갈치 조림 맛있었어요
77862,계절식탁 함덕점,,suj****,뷰가 완전 좋아요 엘리베이터가 있어서 어른과 함께 오기 좋아요 고등어 한상 말고도 ...,4.77,7107.0,뷰 엘리베이터 어른 고등어 많아요
6459,삼대국수회관협재점,"농림축산식품부 제공,안심식당",sto****,국물이 정말 진하고 맛있었어요,NaN,800.0,국물 진하고 맛있었어요
17920,충남식당,제주 성산 갈치조림 진심 맛난 곳,까꿍9184,갈치구이 속살이 너무 부드럽고 고소해 요 겉바 속촉의 정석 갈치구이도 맛있었구요 칼...,NaN,1867.0,갈치 구이 속 부드럽고 고소해 겉바 속 촉 정석 갈치 구이 맛있었구요 칼칼한 갈치 ...
22233,호림식당,"농림축산식품부 제공,안심식당",건민맘84,캐디 소개로 간 집인데 너무 맛있네요 맛집 인증,4.16,739.0,캐디 간 집 맛있네요 맛집 인증
50132,고객식당,제주 동문시장 수요미식회에서 인정한 갈치조림 맛집,딥디끄,나 갈치조림 좋아했었네,4.35,1864.0,갈치 조림 좋아했었네
32545,명랑스낵,,xox88,몇 년 만에 온 건데 그대로에 용 넘존 맛 앉자마자 거의 테이블 풀로 찼어요 맛집 ...,NaN,2738.0,맛 앉자마자 테이블 풀 찼어요 맛집 정띠


In [ ]:
# review_df.to_csv('/content/drive/My Drive/제주도식당리뷰_불용어처리_명사_1.csv', encoding='utf-8-sig', index=False)

### 키워드추출

In [ ]:
# review_df = pd.read_csv('/content/drive/MyDrive/제주도식당리뷰_불용어처리_명사.csv')
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96882 entries, 0 to 96881
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   식당명        96882 non-null  object 
 1   식당정보       96831 non-null  object 
 2   닉네임        96882 non-null  object 
 3   리뷰         96882 non-null  object 
 4   평점         66096 non-null  float64
 5   방문자/블로그리뷰  96831 non-null  float64
 6   review     96882 non-null  object 
dtypes: float64(2), object(5)
memory usage: 5.2+ MB


In [ ]:
review_df.sample(3)

,식당명,식당정보,닉네임,리뷰,평점,방문자/블로그리뷰,review
95212,돈풍년,"농림축산식품부 제공,안심식당",thtnsehd123,맛있어요,NaN,656.0,맛있어요
37837,백기해녀의집,성산일출봉이 보이는 아침식사하기 좋은 성게 칼국수,짱깨비,어르신 모두 친절하시고 반찬 하나 하나 깔끔하고 맛있었어요 맛있게 잘 먹고 성산 쪽...,4.29,1210.0,어르신 친절하시고 반찬 깔끔하고 맛있었어요 맛있게 먹고 성산 산책
17682,마농치킨 5호점,,찾고야만다 맛집,항상 줄 서 있다는 맛집 마농치킨 예전부터 벼르고 벼르다 드디어 맛보았습니다 도대체...,NaN,1013.0,항상 줄 있다는 맛집 마농 치킨 예전 벼르 벼르 맛보았습니다 도대체 일까 먹고 버렸...
5465,삼영식당,제주도 애월 신엄리 숨은 몸국 맛집,하히우헤호,몸국과 고등어구이 맛있게 먹었어요,4.49,378.0,몸국 고등어 구이 맛있게
40621,사계바다 통갈치조림구이 제주산방산점,"농림축산식품부 제공,안심식당",두리둥실빵빵,부모님과의 가족 식사 맛나게 잘 먹었습니다 밑반찬도 맛있고 다들 친절하셔서 즐거운 ...,4.72,4012.0,부모님 가족 맛 밑반찬 맛있고 친절하셔서
57228,성산물고기자리,,임은숙25,맛있어요,4.64,418.0,맛있어요


In [ ]:
# 식당명으로 그룹화하여 리뷰_불용어제거 컬럼의 값을 띄어쓰기로 합치는 함수
def concatenate_reviews(group):
    return ' '.join(group)

# 그룹화하고 띄어쓰기로 합치기
concatenated_reviews = review_df.groupby('식당명')['review'].agg(concatenate_reviews)

In [ ]:
final_review = pd.DataFrame(concatenated_reviews).reset_index()

In [ ]:
final_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   식당명     931 non-null    object
 1   review  931 non-null    object
dtypes: object(2)
memory usage: 14.7+ KB


In [ ]:
# 식당 데이터셋 식당명 中 리뷰 데이터셋에 포함되지 않는 식당 종류
missing_restaurants = jeju_restaurant_df[~jeju_restaurant_df['식당명'].isin(final_review['식당명'])]
missing_restaurants['식당명']

70                   우진사골해장국
65                    올래장터국수
107          한국맥도날드(유)서귀포DT점
33                바다를본돼지서귀포점
7                       남도식당
108                     한남식당
40                     뽈살집본점
41                   뽈살집제주본점
3                    고씨네천지국수
93                      청해식당
95                       칠돈가
29                 모이세해장국중앙점
71                      운산식당
31                     미향해장국
32                  미향해장국동홍점
74                      이한식당
27                      멧돌식당
4                       그때그집
54                      수궁식당
115                     화정식당
52                   솔동산고기국수
45                      삼행식당
103                     평화식당
87                      지정식당
53                      송정식당
77                      정낭식당
79                      정안식당
43                   삼무국수3호점
30                 미풍해장국서귀포점
39                      부두식당
42                삼대국수회관서귀포점
18                      동네식당
19                      동언식당
22                      동호식당
100           

In [ ]:
missing_restaurants = missing_restaurants['식당명'].drop_duplicates()

In [ ]:
excluded_keywords = ['투썸플레이스', 'BHC', '배스킨라빈스','풀바셋','베스킨라빈스','피자헛','페리카나','명륜진사갈비','투썸플레이스','비비큐','폴바셋','도미노피자', '컴포즈커피', '명륜진사갈비', '스타벅스', '파리바게뜨', '맥도날드', '빽다방', '메가커피', '에이바우트', '아웃백', 'KFC', '공차', '롯데리아', '버거킹']

filtered_restaurants = missing_restaurants[~missing_restaurants.str.contains('|'.join(excluded_keywords))]

In [ ]:
len(filtered_restaurants)

485

In [ ]:
filtered_restaurants # missing_restaurants 中 프랜차이즈 매장을 제외한 식당명

70                 우진사골해장국
65                  올래장터국수
33              바다를본돼지서귀포점
7                     남도식당
108                   한남식당
40                   뽈살집본점
41                 뽈살집제주본점
3                  고씨네천지국수
93                    청해식당
95                     칠돈가
29               모이세해장국중앙점
71                    운산식당
31                   미향해장국
32                미향해장국동홍점
74                    이한식당
27                    멧돌식당
4                     그때그집
54                    수궁식당
115                   화정식당
52                 솔동산고기국수
45                    삼행식당
103                   평화식당
87                    지정식당
53                    송정식당
77                    정낭식당
79                    정안식당
43                 삼무국수3호점
30               미풍해장국서귀포점
39                    부두식당
42              삼대국수회관서귀포점
18                    동네식당
19                    동언식당
22                    동호식당
13                    다정이네
16                  돈이레막국수
82               제주산방식당서귀점
81                제주명가두루치기
5

In [ ]:
final_review.sample(10)

,식당명,review
842,한강식당,구수한 된장 맛 독특해요 맛있게 오전 넘어 방문 근처 밥 곳 많지 않더라 평점 괜찮...
579,이모밥상 제주공항본점,이모 밥상 차려진 집밥 밑반찬 간도 맞고 맛있었구요 게장 옥돔 구이 밥 그릇 뚝딱 ...
275,바당길,식전 톳 보리밥 흑 돼지 비빔 장 맛있어요 보말 죽은 말 고소하고 보말도 신선한 많...
468,애월미깡창고,만족스러웠던 카페 서치하자마자 된다 했었는데 왔으면 어쩔 동남아 갬 제대로 야자수 ...
135,더비치크랩 제주공항점,갑각류 먹고 곳 네이버 예약 따로 되니 편하네요 곳 누굴 만족 곳 강력히 드리는 곳...
609,장춘식당,맛 안녕하세요 김 조교 동문 시장 맛집 찾았습니다 뱅기 타러 직전 갔는데요 진정한 ...
405,수선화식당,협재 다니면서 성공한 게 손가락 꼽 수선화 지난달 예약 대기 길어서 사람 많 유가 ...
299,복집식당,갈치 국 적당히 칼칼하면서 시원해서 해장 반찬 간단하고 정갈하게 맛있어서 긁어 재료...
692,제주올래국수,고기국수 땡겨서 근처 간단히 비빔국수 고기 야채 많음 국수 육수 괜찮음 부모님 모시...
837,하영올레국수,지나갈 줄이 길어서 봐야지 방문 되었어요 중국 사람 많았고 국수 메뉴 먹고 있었지만...


## 식당명 전처리

### 리뷰 데이터셋 식당명 jeju_restaurant_df 식당명 형식에 맞게 수정

In [ ]:
# 식당명 형식 맞추기 (1)
final_review['식당명'] = final_review['식당명'].replace({'제주도 고기국수 만세국수': '제주도고기국수만세국수', '모이세해장국 연동점': '모이세해장국연동점', '가시식당 2호점': '가시식당2호점', '은희네해장국 이도점': '은희네해장국이도점', '애월연어 시청점': '애월연어시청점', '애월연어 제주함덕점': '애월연어함덕점', '김영미제첩해장국': '김영미재첩해장국', '안다미로 늘봄 흑염소': '안다미로늘봄', '계절식탁시청점': '계절식탁', '일품순두부 시청점': '일품순두부시청점', '샤브마니아 제주아라점': '샤브마니아제주아라점', '제주정낭집 이도점': '제주정낭집이도점', '스시도모다찌 제주점': '스시도모다찌제주점', '고집돌우럭 제주함덕점': '고집돌우럭함덕점', '함골식당 함덕본점': '함골식당', '함덕골목 본점': '함덕골목', '갈치옥 함덕': '갈치옥', '해오반 함덕점': '해오반함덕점', '담백 함덕점': '제주담백', '명리동식당 애월점': '명리동식당애월점', '삼대국수회관 신제주점': '삼대국수회관신제주점', '논짓물식당': '논짓물', '황금돈가 연동 흑돼지': '황금돈가', '이모밥상 제주공항본점': '이모밥상', '원조 삼성혈해물탕 본점': '삼성혈해물탕', '김재훈고사리육개장': '훈이네고사리육개장', '대춘해장국 시청점': '대춘해장국시청점', '순풍해장국 이도점': '순풍해장국이도점', '산방식당 제주점': '산방식당제주점', '에이바우트커피 구남점': '에이바우트커피구남점', '뚱딴지 애월본점': '뚱딴지', '애월 삼육공본점 숙성흑돼지': '애월삼육공', '애월전분공장카페': '애월전분공장', '애월 우니담': '애월우니담', '함덕회춘': '회춘', '섬오름 해물라면': '섬오름해물라면', '깡촌흑돼지 함덕본점': '깡촌흑돼지', '함덕 보배쌤보리김치와게장': '보배쌤보리김치와게장', '화로위 함덕해수욕장점': '화로위', '제주돈 애월점': '제주돈애월점', '애월찜 애월점': '애월찜', '은희네해장국 2호점': '은희네해장국2호점', '순수한둠비 연동점': '순수한둠비연동점', '대춘해장국 노형점': '대춘해장국노형점', '모이세해장국 연동점': '모이세해장국연동점', '신제주보말칼국수 제주본점': '신제주보말칼국수제주본점', '미풍해장국 연동점': '미풍해장국연동점', '대도식당 연동점': '대도식당연동점', '소낭 비프 하우스 제원': '소낭', '명륜진사갈비 제주연동점': '명륜진사갈비제주연동점', '돈풍 노형흑돼지': '돈풍', '제주부엌 고기국수 제주공항점': '제주부엌', '하여금 노형본점': '하여금', '빠빠라기 신제주점': '빠빠라기신제주점', 'project064': '프로젝트064', '제주세호해장국 연동직영점': '제주세호해장국연동점', '제주세호해장국 본점': '제주세호해장국제주본점', '제주오성 갈치조림 제주도 중문 본점': '제주오성', '돌카롱 중문점': '돌카롱중문점', '명호마농갈비 중문점': '명호마농갈비중문점', '중문회포장센터 새벽야시장': '중문회포장센터새벽야시장', '용이식당 중문점': '용이식당중문점', '제주미향 중문본점': '제주미향', '천돈가 중문본점': '천돈가중문본점', '산지해장국 중문점': '산지해장국중문점', '두리둠비 제주서귀포본점': '두리둠비', '로그릭 중문점': '로그릭중문점', '칠돈가 표선직영점': '칠돈가표선직영점', '표선 돌담칼국수': '돌담칼국수', '국수앤 표선칼국수': '국수앤표선칼국수', '표선자연산횟집 다미진': '다미진횟집', '제주흑돈세상수라간 표선점': '제주흑돈세상수라간표선점', '숙성도 중문점': '숙성도중문점', '국수바다 서귀포본점': '국수바다본점', '중문색달통갈치 본점': '중문색달통갈치', '덤장 중문점': '덤장중문점', '중문해물라면오빠네 본점': '중문해물라면오빠네본점', '미향해장국 중문점': '중문미향해장국본점', '중문우동한그릇': '한그릇', '제주스럽닭 본점': '제주스럽닭', '제주고사리해장국 중문본점': '제주고사리해장국중문본점', '맛존디 국수': '맛존디', '칠돈가 중문점': '칠돈가직영점', '제주중문횟집 퍼랭': '퍼랭', '착한전복 제주중문점': '착한전복제주중문점', '정직한돈 서귀포중문점': '정직한돈중문점', '정직한통닭 중문점': '정직한통닭중문점', '오늘의바다': '카페오늘의바다', '불턱버거': '불턱버거클럽', '제주중문횟집 스시망고': '스시망고', '돈사돈 중문점': '돈사돈중문점', '소담명가 찰솥밥 본점': '소담명가찰솥밥', '한라산아래첫마을 제주민속촌점': '한라산아래첫마을표선민속촌점', '육떡식당 문어떡볶이 본점': '육떡식당', '삼강식당 중문점': '삼강식당중문점', '카페 귤꽃다락': '카페귤꽃다락', '삼성혈해물탕 서귀포점': '삼성혈해물탕서귀포점', '장가네일품순두부 신시가지점': '일품순두부신시가지점', '형제도식당 본점': '형제도식당', '도두반점 호텔더본점': '도두반점호텔더본점', '중문횟집 엉덩물횟집': '엉덩물횟집', '본가 호텔더본제주점': '호텔더본제주', '맛있수다 중문본점': '맛있수다', '다다익고 정육점식당 호텔더본점': '다다익고정육점식당', '중문생선구이 중문본점': '중문생선구이', '숙성돈가 제주중문본점': '숙성돈가', '만족한상회 중문점': '만족한상회', '쉬멍해장국 중문점': '쉬멍해장국중문점', '장가네 해장국': '장가네해장국', '고집돌우럭 중문점': '고집돌우럭중문점', '중문 가람돌솥밥': '중문가람돌솥밥'})

In [ ]:
# 식당명 형식 맞추기 (2)
final_review['식당명'] = final_review['식당명'].replace({'식당 마요네즈':'식당마요네즈','섬타르 제주공항점':'섬타르','미국식 제주점':'미국식제주점','칠돈가 본점':'칠돈가본점','보래드 베이커스':'보레드베이커스','윤옥 본점':'윤옥', '다올국수':'다올국수 제주시','장인의집 제주탑동점':'용담장인의집','도토리키친 본점':'도토리키친','스시도모다찌 칠성점':'스시도모다찌칠성점','동빵 제주동문시장점':'동빵제주동문시장점','고국수 공항본점':'고국수','오현불백 제주 동문시장점':'오현불백','국수마당 본점':'국수마당본점','삼대국수회관 본점':'삼대국수회관본점','도라지식당 제주공항본점':'도라지식당','물항식당 탑동점':'물항식당탑동점',
                                                                     '고집돌우럭 제주공항점':'고집돌우럭','미풍해장국 제주항점':'미풍해장국제주항점','국시트멍 제주고기국수':'국시트멍','노형삼대국수':'삼대국수회관노형점','애월 해성도뚜리':'해성도뚜리','바삭돈가스 애월해안도로점':'바삭애월해안도로점','갈치관 애월해안도로점':'갈치관','하영칼국수 애월본점':'하영칼국수','우심 제원노형':'우심제주점','피즈 버거 노형점':'피즈노형','애월연어 노형점':'애월연어노형점','금갈치 동문시장점':'금갈치','돌카롱 제주공항점':'돌카롱제주공항점','정가네식당':'정가네','제주 고기국수 만세국수':'제주고기국수만세국수','제주고집 제주공항점':'제주고집',
                                                                     '제주공항그때그집':'제주공항그때그집직영점','제주어멍 통갈치 제주공항 본점':'제주어멍통갈치','함쉐프키친 노형점':'함쉐프키친시티점','산지해장국 노형직영점':'산지해장국노형직영점','모이세해장국 노형점':'모이세해장국노형점','카페 109':'카페109', '오쿠다 공항점':'오쿠다공항점','제주김만복김밥 본점':'제주김만복본점','덕인당보리빵 제주점':'덕인당보리빵제주점','바이러닉 에스프레소 바 제주점':'바이러닉에스프레소바','제주태백산 본점':'태백산제주본점','황금손가락 삼화점':'황금손가락삼화점','산지해장국 삼양점':'산지해장국삼양점','하귀 보배쌤보리김치와게장':'보배쌤보리김치와게장하귀점',
                                                                     '애월흑돼지 하귀 흑생':'흑생','바다속고등어쌈밥 제주애월본점':'바다속고등어쌈밥','커피냅로스터스 제주':'커피냅로스터스제주','제주할망밥상 애월점':'제주할망밥상애월점','원조 삼성혈해물탕 1호점':'삼성혈해물탕1호점','예소담 도두점':'도두예소담','삼미횟집 제주공항점':'삼미횟집','뉴욕버거 제주노형점':'뉴욕버거제주노형점','돈사돈 별관':'돈사돈','제주 하멜':'제주하멜','샤브마니아 제주노형점':'샤브마니아제주노형점','한림칼국수 제주공항점':'한림칼국수제주공항점','앙뚜아네트':'앙뚜아네트용담점','용두암 해녀촌':'용두암해녀촌','용두암 엄마 해장국':'용두암엄마해장국','해월정 제주공항점':'해월정제주공항점',
                                                                     '카페나모나모베이커리':'카페나모나모','애월미깡창고':'미깡창고','카페리 구좌 카페 평대리해변 세화 소품':'카페리','구좌 만물식당 월정리세화':'만물식당','한림국수 집':'한림국수집','면뽑는선생만두빚는아내 제주협재점':'면뽑는선생만두빚는아내','ODE Music Bar':'오드','애월당 제주공항점':'애월당공항점','리치망고 애월본점':'리치망고애월본점','서진향해장국 성산점':'서진향해장국성산점','코코마마 성산점':'코코마마성산점','성산바다풍경 제주성산일출봉점':'성산바다풍경','금돗 성산흑돼지':'금돗','마농치킨 5호점':'성산포마농치킨5호점','진미식당 성산일출봉점':'진미식당','선미식당 제주성산일출봉점':'선미식당',
                                                                     '전망좋은횟집&흑돼지 성산본점':'전망좋은횟집','바다를본돼지 서귀포올레시장점':'바다를본돼지서귀포점','어멍뚝배기 올레시장본점':'어멍뚝배기','올래국수 서귀포점':'올래국수서귀포점','낭쿰낭쿰 올레시장본점':'낭쿰낭쿰','고씨네천지국수 올레시장본점':'고씨네천지국수','다정이네 올레시장 본점':'다정이네','송정식당 갈치조림구이 서귀포올레시장본점':'송정식당', '하르비커피':'하루비커피','카페블루하우스 서귀포점':'카페블루하우스','먹고정 서귀포올레시장본점':'먹고정','양평해장국 서귀포점':'양평해장국','돈사향 올레시장본점':'돈사향서귀포점','솔동산고기국수 서귀포올레시장본점':'솔동산고기국수',
                                                                     '미풍해장국 서귀포점':'미풍해장국서귀포점','삼대국수회관 서귀포직영점':'삼대국수회관서귀포점','운산장어구이':'운산식당','뽈살집 본점':'뽈살집본점','평화식당 서귀포점':'평화식당','미향해장국 본점':'미향해장국','모이세해장국 중앙점':'모이세해장국중앙점','칠돈가 서귀포점':'칠돈가','미풍해장국 서귀포직영점':'미풍해장국서귀포직영점','삼강식당 본점':'삼강식당','정이가네':'정이가네본점','제주등대아구찜 서귀포점':'제주등대아구찜서귀포점','제주명가두루치기 서귀포올레시장점':'제주명가두루치기','문치비 서귀포신시가지점':'문치비본점','한스옥':'마담조','화고 흑돼지 신시가지점':'제주화고',
                                                                     '앙끄레국수 서귀포본점':'앙끄레국수','산지해장국 서귀포점':'산지해장국서귀포점','제주산방식당 함덕점':'산방식당함덕점','함덕해물라면오빠ZIP':'함덕해물라면오빠네','해녀김밥 본점':'해녀김밥본점','숙성도 함덕점':'숙성도함덕점','원풍해장국 함덕점':'원풍해장국','제주순풍해장국 함덕점':'제주순풍해장국','별미해장국 본점':'별미해장국본점','칠돈가 함덕 흑돼지':'칠돈가함덕점','카페서연의집':'서연의집','섬소나이 위미점':'섬소나이위미점','애월 고불락':'고불락','제주김만복 애월점':'제주김만복애월점','제주 정직한돈 애월점':'정직한돈본점','안녕협재씨 제주협재점':'안녕협재씨제주협재점','제주협재 바다해찬':'제주바다해찬',
                                                                     '해물통라면 문개항아리 애월해안도로점':'문개항아리애월점','제주광해 애월':'제주광해애월','금돈가 제주 애월해안도로 본점':'금돈가','회춘 애월점':'회춘애월점','회가서쪽에서뜨겠네 애월':'회가서쪽에서뜨겠네','장인의집[본점]':'장인의집','애월 갈치 암행어사':'어박사애월점','제주도 해녀세자매 협재본점':'해녀세자매','미풍해장국 한림점':'미풍해장국한림점','한라축산정육식당 제주흑돼지한우 한림협재애월점':'한라축산정육식당','파스테이스 제주점':'파스테이스','뼈대감 본점':'뼈대감','도민상회본점 제주한림협재흑돼지':'도민상회본점한림협재','옥만이네 협재본점':'옥만이네','버거요 제주본점':'버거요제주본점'})

In [ ]:
# 식당명 형식 맞추기 (3)
final_review['식당명'] = final_review['식당명'].replace({'제주로운청해원 성산일출봉점':'제주로운청해원','제주마루 섭지코지점':'제주마루','섭지해녀의 집':'섭지해녀의집','제주일도 서귀포 성산점':'제주일도','삼미흑돼지 중문점':'삼미흑돼지','까망돼지 서귀포점':'까망돼지','중문통갈치조림구이 기원은갈치':'기원은갈치','중문 돌담흑돼지':'돌담흑돼지중문','몬스터살롱 애월':'몬스터살롱','애월빵공장 앤 카페':'애월빵공장&카페','함쉐프키친 서귀포 본점':'함쉐프키친','황해식당 제주공항 갈치':'황해식당','니모메':'니모메빈티지라운지','성산일출봉 어멍횟집':'성산일출봉어멍횟집','미향해장국 성산점':'미향해장국성산점','삼무국수 2호점':'삼무국수2호점',
                                                                     '성산포 자연산 회센타':'성산포자연산회센타','꽃가람 제주성산일출봉점':'꽃가람','성산카페 호랑호랑':'성산호랑호랑','미향해장국 동홍점':'미향해장국동홍점','이스틀리카페':'이스틀리','제주어물전 성산본점':'어물전','어조횟집 성산일출봉점':'성산어조횟집','섭지코지로 성산본점':'섭지코지로','동도원 중문점':'동도원','서울앵무새 제주':'서울앵무새','다정이네김밥 서귀포신시가지점':'다정이네김밥서귀포신시가지점','돈사돈 서귀포점':'돈사돈서귀포점','사계바다 통갈치조림구이 제주산방산점':'사계바다','올레마당 제주산방산점':'올레마당','해녀밥상 제주산방산점':'해녀밥상','코데인커피로스터스 산방산점':'코데인커피로스터스',
                                                                     '젠하이드어웨이 제주점':'젠하이드어웨이제주점','환이네 이태리식당':'환이네이태리식당','춘미향음식점':'춘미향','제주산방산나들목':'산방산나들목','훼미리맛짱분식':'맛짱분식','중앙식당':'중앙간이식당','고은물식당 제주산방산점':'고은물식당','순심이네한우흑돼지':'순심이네','러디스 서귀포본점':'러디스','제주김만복 서귀포점':'제주김만복김밥서귀포점','카페루시아 본점':'카페루시아','명호돗갈비 대평본점':'명호돗갈비','동복해녀식당 구좌본점':'해녀식당','미영이네':'미영이네식당','모슬포한라전복 본점':'모슬포한라전복','제주할망밥상 모슬포점':'제주할망밥상모슬포점','미풍해장국 모슬포점':'미풍해장국모슬포점',
                                                                     '모이세해장국모슬포점':'모이세해장국모습포점','신옛촌 산방산점':'신옛촌본점','거멍국수 제주산방산점':'거멍국수','사계전복 산방산본점':'사계전복','미향해장국 남원점':'미향해장국남원점','표선세화해녀의집':'세화리해녀의집','미향해장국 표선점':'미향해장국표선점','탐라우돈정육식당':'탐라우돈','제주만석밀냉면':'제주만석','고기왕 하귀애월점':'고기왕','망고홀릭 애월해안도로점':'망고홀릭애월해안도로점','그리운바다성산포 성산본점':'그리운바다성산포','성산흑돼지 흑돈쭐':'성산흑돈쭐별관','흑돈쭐 제주성산일출봉점':'성산흑돈쭐','마농치킨 중앙통닭 2호점':'마농치킨중앙통닭2호점','삼무국수 서귀포3호점':'삼무국수3호점',
                                                                     '강실장회포차':'강실장실내회포차','곱들락 함덕흑돼지':'곱들락','로맨틱새우 애월곽지본점':'로맨틱새우','돈이랑 성산점':'돈이랑성산점','보리타작 제주산방산점':'보리타작','산방흑돼지순대본점':'산방흑돼지순대','전복죽일번가 애월해안도로본점':'전복죽일번가','마루해물라면 제주공항점':'마루해물','돌담너머바다 & 현무암닭강정':'현무암닭강정','꽃길국수 공항본점':'꽃길식당용담본점','숙자네숟가락젓가락 구좌읍세화본점':'숙자네숟가락젓가락','제갈양 제주협재점':'제갈양제주협재점','성산물고기자리':'물고기자리','산지해장국 성산점':'산지해장국성산점','오랑우탄면사무소 혁신도시점':'오랑우탄면사무소신서귀포점','제주갈치맛집 영목이네 표선점':'제주갈치맛집영목이네',
                                                                     '장승포식당 성산 서귀포점':'장승포식당','망고홀릭 종달점':'망고홀릭성산점','독개물항 제주한림 협재 본점':'독개물항','해송수산':'해송횟집','하르방밀면 노형점':'하르방밀면','영천횟집 탑동본점':'영천횟집','통큰장어 신제주점':'제주통큰장어별관','오누이찐빵':'오누이','영복갈비 제주본점':'영복갈비제주본점','애월회포장센터 애월밤바다':'제주횟집애월밤바다','제주인바다 제주공항점':'제주인바다','해송갈치 제주성산일출봉점':'성산일출봉해송갈치전문','솔솥 서귀포점':'솔솥서귀포점','작산 애월흑돼지':'작산','자연과사람들 밀면':'자연과사람들밀면','휴즐리 제주':'휴즐리','노도 고등어회':'노도고등어회',
                                                                     '황금손가락 용담점':'황금손가락용담점','먹쿠슬낭카페':'먹쿠슬낭여행자카페','제주 도도리고기국수':'도도리고기국수','신해바라기분식 한라대점':'신해바라기분식한라대점','옹포바다횟집':'용포횟집','성산을본고등어쌈밥 제주성산일출봉점':'성산을본고등어쌈밥제주성산일출봉점','제주산방식당 서귀점':'제주산방식당서귀점','그랜마스 서귀포시청점':'그랜마스서귀포시청점','제주산방식당 협재점':'제주산방식당협재점','집의기록상점':'집의기록1','제주산방식당 외도점':'제주산방식당외도점','큰갯물횟집 중문본점':'큰갯물횟집','영웅식당 제주 산방산점':'영웅식당','카페 가드니아':'카페가드니아','동진식당 공항점':'동진식당공항점',
                                                                     '콩이네두부':'콩이네','제주올레면옥 제주본점':'제주올레냉면제주점','백록집 제주함덕점':'백록집','애월포구횟집':'포구횟집','남춘식당':'난춘식당','순이네':'순이식당','해비치불턱카페':'해비치불턱','24시전주명가콩나물국밥':'시전주명가콩나물국밥노형점','화석과자 센잇':'센잇','더비치크랩 제주공항점':'더비치크랩제주공항점','꽃가마':'꽃가마식당','24시전주명가콩나물국밥 일도점':'24시전주명가콩나물국밥일도점','다다익고 정육식당 제주사수점':'다다익고정육식당신서귀포점','망고홀릭 제주본점':'망고홀릭제주본점','백년해장국 표선성읍점':'백년해장국성산점','산지해장국 대정점':'산지해장국대정점','태연종합정육식당 제주공항점':'종합정육식당',
                                                                     '함덕카페오다':'카페오다','갈치요리 전문점 뜰에':'뜰에','풍로 제주신화월드점':'풍로제주신화월드','산지해장국 외도점':'산지해장국외도점','무한의 서 커피로스터스':'무한의서커피로스터스','하토우 커피 로스터스':'하토우커피로스터스','무슈부부커피스탠드 관덕정':'무슈부부커피스탠드관덕정','블루메베이글 노형점':'블루메베이글노형점','오직흑돼지 제주공항본점':'오직흑돼지','해일리 카페':'해일리','비주비주 한림점':'비주비주한림점'})

In [ ]:
# jeju_restaurant_df.to_csv('/content/drive/My Drive/제주숙박식당정보_최종본_1.csv', encoding='utf-8-sig', index=False)

## Counter 적용

In [ ]:
final_review.sample(10)

,식당명,review
371,석이네식당,두루치기 맛 불친절해요 그릇 탁탁 내려놓고 밥 가지 오셨을 손 반찬 그릇 옮기고 있...
157,돈블럭,인생 돼지고기 넣자마자 탄성 나옵니다 찌개 냉면 반찬 맛있어요 구워 주셔 알맞게 익...
67,금능식당,고사리 육계장 몸국 성게 국수 생소하고 맛 궁금해서 맛보러 학교 급식 나올 만큼 친...
900,호도제과,방문 첫 남겨요 재료 가득한 호도 제과 맛 보신다면 빵 순례 코스 들어가는 되지 않...
138,더힐링타임,한라산 등산 사우나 싶었지만 몸 컨디션 아쉬운데로 족 욕 예약 방문 주차장 편하구 ...
64,그시절그짬뽕,소문 듣고 서울 출발 가장 곳 고추 짬뽕 간 짬뽕 친절하고 정성 다해 부부 힘드실까...
853,한스시,리조트 근처 스시 초밥 맛집 하나로마트 주차 깔끔하고 회도 두툼 싱싱해요 아주머니 ...
675,제주부엌,부엌 고기국수 찐 맛집 공항 전이 비빔국수 국물 떠 맛있네요 순대 돔베 편육 세트 ...
516,오또도넛,크리스 피 도넛 도넛 능가 맛 유명한 랜 디스 도넛 맛있어요 근처 으면 곳 남편 아...
661,제주만주,택배 가능한 디저트 찾으시면 부드러운 수제 파이 들어가는 앙금 대박 않으 면서 밤 ...


In [ ]:
from collections import Counter

# '리뷰_불용어제거' 컬럼에서 문자열을 띄어쓰기로 분리하여 리스트로 변환 & 상위 30개의 단어 선택하여 하나의 문자열로 만들기
final_review['review_keyword'] = final_review['review'].apply(lambda x: ' '.join([word[0] for word in Counter(x.split()).most_common(30)]))

final_review.head(3)

,식당명,review,review_keyword
0,시전주명가콩나물국밥노형점,오랜 갔더니 돈까스 생겼네요 크기 크고 맛 친절하시고 돈까스 크게 칼 자르실 같다구...,돈까스 콩나물 국밥 맛있어요 친절하고 콩나물국 계란 말 그러더니 오랜 갔더니 생겼네...
1,24시전주명가콩나물국밥일도점,짝꿍 양 꼬치 집 자리 되돌아오는 길 콩나물 국밥집 추운데 뚝배기 하실까 옴 콩나물...,맛있어요 맛 국밥 콩나물 가격 저렴하고 밥 돈까스 곳 맛있고 말 맛있음 단골 국물 ...
2,513텐동,우연 주변 업무 차 동생 오픈 런 텐동 바삭 튀김 맛 일품 연어 텐동 육회 텐동 동...,텐동 맛 튀김 맛집 뷰 바다 먹고 집 맛있는 바삭 곳 호 해수욕장 동생 개업 동 유...


In [ ]:
from collections import Counter
# '리뷰_불용어제거' 컬럼의 모든 행 데이터 합치기
merged_text = ' '.join(review_df['review'])

# 띄어쓰기를 기준으로 단어 분할 및 빈도수 계산
word_counts = Counter(merged_text.split()).most_common(632)

In [ ]:
# 빈도수가 200개 이상인 단어만 출력
filtered_top_words = [(word, count) for word, count in word_counts if count >= 200]

In [ ]:
filtered_top_words

[('맛', 25821),
 ('맛있어요', 20571),
 ('고기', 12736),
 ('갈치', 12222),
 ('맛집', 12007),
 ('방문', 10659),
 ('먹고', 10452),
 ('맛있게', 9976),
 ('곳', 9537),
 ('맛있고', 7416),
 ('조림', 7318),
 ('뷰', 6703),
 ('반찬', 5626),
 ('밥', 5342),
 ('맛있었어요', 5253),
 ('회', 5130),
 ('구이', 5011),
 ('집', 4739),
 ('고등어', 4338),
 ('가격', 4218),
 ('돼지', 4132),
 ('흑', 4121),
 ('친절하시고', 4023),
 ('메뉴', 3903),
 ('전복', 3894),
 ('국물', 3849),
 ('게', 3645),
 ('양', 3517),
 ('국수', 3398),
 ('바다', 3311),
 ('근처', 3237),
 ('맛있어서', 3127),
 ('많고', 3114),
 ('분위기', 3095),
 ('맛있는', 2933),
 ('커피', 2858),
 ('배', 2850),
 ('푸', 2824),
 ('맛있네요', 2721),
 ('카페', 2677),
 ('해장국', 2603),
 ('말', 2573),
 ('가족', 2526),
 ('양념', 2517),
 ('밑반찬', 2418),
 ('맛있습니다', 2292),
 ('깔끔하고', 2273),
 ('서비스', 2257),
 ('앞', 2090),
 ('가게', 2084),
 ('신선하고', 2082),
 ('세트', 1943),
 ('김치', 1877),
 ('주차', 1848),
 ('아침', 1845),
 ('짐', 1844),
 ('칼국수', 1819),
 ('튀김', 1803),
 ('해물', 1803),
 ('사람', 1798),
 ('미역국', 1740),
 ('성게', 1675),
 ('부모님', 1645),
 ('좋았습니다', 1630),
 ('역시', 1626),

In [ ]:
# '리뷰_키워드_최종' 컬럼 추가
final_review['review_keyword_final'] = final_review['review_keyword'].apply(lambda x: [word for word, _ in filtered_top_words if word in x])

In [ ]:
final_review['review_keyword_final'].sample(3)

351    [맛, 맛있어요, 고기, 맛집, 방문, 먹고, 곳, 맛있고, 밥, 회, 집, 돼지,...
661    [맛, 방문, 곳, 집, 가격, 커피, 배, 가지, 향, 냄새, 애월, 면, 디저트...
887    [맛, 맛집, 맛있고, 밥, 회, 집, 친절하시고, 게, 많고, 맛있습니다, 가게,...
Name: review_keyword_final, dtype: object

In [ ]:
len(final_review['식당명'].unique())

931

In [ ]:
final_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   식당명                   931 non-null    object
 1   review                931 non-null    object
 2   review_keyword        931 non-null    object
 3   review_keyword_final  931 non-null    object
dtypes: object(4)
memory usage: 29.2+ KB


In [ ]:
# final_review.to_csv('/content/drive/My Drive/제주도식당리뷰_최종본_1.csv', encoding='utf-8-sig', index=False)

# 정리

> 'WE 호텔 제주', '베스트웨스턴 제주', '엠버리조트', '호텔 토스카나', '켄싱턴리조트 제주중문' (치완님 호텔 中 없는 호텔) 총 5곳

In [ ]:
# final_review = pd.read_csv('/content/drive/MyDrive/제주도식당리뷰_최종본.csv')
final_review.sample(3)

,식당명,review,review_keyword,review_keyword_final
176,동백국수,아침 비행기 근처 일찍 곳 찾아보니 숙소 앞 맛집 있었네 예쁘고 정원 멋있는 곳 식...,국수 고기 공항 맛집 동 맛 근처 곳 맛있고 고기국수 동백 정원 아침 앞 양 친절하...,"[맛, 고기, 맛집, 방문, 곳, 맛있고, 뷰, 집, 국물, 양, 국수, 근처, 앞..."
587,일억조,본적 고소하면서 시원하고 맛있옸습니 밑반찬 맛있었어요 맛있었습니다 친절하셔서 유튜버...,맛 뼈국 국물 차돌 맛있어요 접짝 맛집 뼈 먹고 찌개 장 고기 방문 맛있게 맛있었어...,"[맛, 맛있어요, 고기, 맛집, 방문, 먹고, 맛있게, 맛있고, 반찬, 밥, 맛있었..."
553,우심제주점,여행가 흑 돼지 마카 가려다가 포기 간 곳 가길 했단 한우 웻 이징 퀄리티 미쳤는데...,고기 소고기 맛 한우 맛집 맛있어요 분위기 근처 공항 육회 맛있게 먹고 곳 소금 형...,"[맛, 맛있어요, 고기, 맛집, 방문, 먹고, 맛있게, 곳, 맛있고, 반찬, 회, ..."


In [ ]:
len(final_review['식당명'].unique())

931

In [ ]:
len(jeju_restaurant_df['식당명'].unique())

972

In [ ]:
# '식당명'이 final_review에는 있지만 jeju_restaurant_df에 없는 데이터 추출
not_included = final_review[~final_review['식당명'].isin(jeju_restaurant_df['식당명'])]

len(not_included['식당명'].unique())

239

In [ ]:
not_included.head()

,식당명,review,review_keyword,review_keyword_final
0,시전주명가콩나물국밥노형점,오랜 갔더니 돈까스 생겼네요 크기 크고 맛 친절하시고 돈까스 크게 칼 자르실 같다구...,돈까스 콩나물 국밥 맛있어요 친절하고 콩나물국 계란 말 그러더니 오랜 갔더니 생겼네...,"[맛, 맛있어요, 밥, 친절하시고, 국물, 게, 양, 맛있어서, 많고, 해장국, 말..."
1,24시전주명가콩나물국밥일도점,짝꿍 양 꼬치 집 자리 되돌아오는 길 콩나물 국밥집 추운데 뚝배기 하실까 옴 콩나물...,맛있어요 맛 국밥 콩나물 가격 저렴하고 밥 돈까스 곳 맛있고 말 맛있음 단골 국물 ...,"[맛, 맛있어요, 곳, 맛있고, 밥, 집, 가격, 메뉴, 국물, 게, 배, 말, 김..."
2,513텐동,우연 주변 업무 차 동생 오픈 런 텐동 바삭 튀김 맛 일품 연어 텐동 육회 텐동 동...,텐동 맛 튀김 맛집 뷰 바다 먹고 집 맛있는 바삭 곳 호 해수욕장 동생 개업 동 유...,"[맛, 맛집, 방문, 먹고, 곳, 뷰, 밥, 회, 집, 바다, 맛있는, 말, 튀김,..."
5,BBQ치킨 제주협재해수욕장점,메뉴 천원 올려 받음 메뉴판 없는데 가격 맞춘다고 시키지 스파클링 꾸역꾸역 천원 추...,치킨 서비스 비비큐 맛있게 맛 맛있어요 앞 치즈볼 천원 숙소 건물 집 피자 맛있네요...,"[맛, 맛있어요, 방문, 맛있게, 집, 게, 맛있네요, 말, 서비스, 앞, 개, 맛..."
9,가마리식당,작성 해달라고도 하셨는데 가보셨으면 적습니다 맛있어요 고기 이런지 반찬 맛있어서 볶...,맛있어요 볶음밥 숙소 맛집 고기 맛 집 먹고 근처 작성 해달라고도 하셨는데 가보셨으...,"[맛, 맛있어요, 고기, 맛집, 먹고, 반찬, 밥, 집, 게, 근처, 맛있어서, 앞..."


In [ ]:
# '식당명' 컬럼을 기준으로 merge 수행
final_df = pd.merge(final_review, jeju_restaurant_df, on='식당명', how='inner')

# 병합된 데이터프레임의 식당명(unique한 값) 개수 출력
print("final_review와 jeju_restaurant_df 공통으로 등장하는 식당명의 개수:", len(final_df['식당명'].unique()))

final_review와 jeju_restaurant_df 공통으로 등장하는 식당명의 개수: 692


In [ ]:
len(final_df['숙박업명'].unique())

91

In [ ]:
final_df.drop(['review','review_keyword','숙박업주소','식당주소'],axis=1, inplace=True)
final_df.rename(columns={'review_keyword_final':'리뷰_키워드'},inplace=True)

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   식당명           2889 non-null   object 
 1   리뷰_키워드        2889 non-null   object 
 2   숙박업명          2889 non-null   object 
 3   도시명           2889 non-null   object 
 4   숙박위도          2889 non-null   float64
 5   숙박경도          2889 non-null   float64
 6   대분류명          2889 non-null   object 
 7   식당위도          2889 non-null   float64
 8   식당경도          2889 non-null   float64
 9   검색량합계값        2889 non-null   int64  
 10  숙박_식당 거리(km)  2889 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 248.4+ KB


최종 숙박업 갯수: 88개
최종 음식점 갯수: 331개

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/제주도식당리뷰_불용어처리_명사_1.csv')
test_df.drop(['닉네임','리뷰','review'],axis=1,inplace=True)
test_df = test_df.drop_duplicates()
test_df.head()

,식당명,식당정보,평점,방문자/블로그리뷰
0,만강촌옛날칼국수,NaN,4.33,1309.0
10,제주돔베고기집,NaN,4.48,3935.0
20,미스칠,"농림축산식품부 제공,안심식당",4.33,1986.0
288,식당 마요네즈,NaN,4.57,2577.0
298,섬타르 제주공항점,제주공항근처카페 타르트가 맛있는 감성카페,4.72,3906.0


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   식당명           2889 non-null   object 
 1   리뷰_키워드        2889 non-null   object 
 2   숙박업명          2889 non-null   object 
 3   도시명           2889 non-null   object 
 4   숙박위도          2889 non-null   float64
 5   숙박경도          2889 non-null   float64
 6   대분류명          2889 non-null   object 
 7   식당위도          2889 non-null   float64
 8   식당경도          2889 non-null   float64
 9   검색량합계값        2889 non-null   int64  
 10  숙박_식당 거리(km)  2889 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 248.4+ KB


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 935 entries, 0 to 96645
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   식당명        935 non-null    object 
 1   식당정보       415 non-null    object 
 2   평점         655 non-null    float64
 3   방문자/블로그리뷰  928 non-null    float64
dtypes: float64(2), object(2)
memory usage: 36.5+ KB


In [ ]:
final_df = pd.merge(final_df, test_df, on='식당명', how='left')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2898 entries, 0 to 2897
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   식당명           2898 non-null   object 
 1   리뷰_키워드        2898 non-null   object 
 2   숙박업명          2898 non-null   object 
 3   도시명           2898 non-null   object 
 4   숙박위도          2898 non-null   float64
 5   숙박경도          2898 non-null   float64
 6   대분류명          2898 non-null   object 
 7   식당위도          2898 non-null   float64
 8   식당경도          2898 non-null   float64
 9   검색량합계값        2898 non-null   int64  
 10  숙박_식당 거리(km)  2898 non-null   float64
 11  식당정보          898 non-null    object 
 12  평점            1328 non-null   float64
 13  방문자/블로그리뷰     1757 non-null   float64
dtypes: float64(7), int64(1), object(6)
memory usage: 317.1+ KB


In [ ]:
# 컬럼명 및 순서 정리
final_df = final_df[['식당명','식당정보','대분류명','도시명','식당위도','식당경도','숙박업명','숙박위도','숙박경도','숙박_식당 거리(km)','평점','방문자/블로그리뷰','검색량합계값','리뷰_키워드']]

In [ ]:
# NaN 또는 무한 값을 0으로 대체
final_df['방문자/블로그리뷰'].fillna(0, inplace=True)
final_df['방문자/블로그리뷰'] = final_df['방문자/블로그리뷰'].replace([np.inf, -np.inf], 0)

# 다시 정수로 변환
final_df['방문자/블로그리뷰'] = final_df['방문자/블로그리뷰'].astype('int')

In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2898 entries, 0 to 2897
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   식당명           2898 non-null   object 
 1   식당정보          898 non-null    object 
 2   대분류명          2898 non-null   object 
 3   도시명           2898 non-null   object 
 4   식당위도          2898 non-null   float64
 5   식당경도          2898 non-null   float64
 6   숙박업명          2898 non-null   object 
 7   숙박위도          2898 non-null   float64
 8   숙박경도          2898 non-null   float64
 9   숙박_식당 거리(km)  2898 non-null   float64
 10  평점            1328 non-null   float64
 11  방문자/블로그리뷰     2898 non-null   int64  
 12  검색량합계값        2898 non-null   int64  
 13  리뷰_키워드        2898 non-null   object 
dtypes: float64(6), int64(2), object(6)
memory usage: 317.1+ KB


In [ ]:
final_df.sample(3)

,식당명,식당정보,대분류명,도시명,식당위도,식당경도,숙박업명,숙박위도,숙박경도,숙박_식당 거리(km),평점,방문자/블로그리뷰,검색량합계값,리뷰_키워드
2645,하여금,NaN,한식,제주시,33.485107,126.482509,노형호텔,33.486122,126.478205,0.415,NaN,0,535,"맛, 맛있어요, 고기, 맛집, 방문, 먹고, 맛있게, 곳, 맛있고, 반찬, 맛있었어..."
305,논짓물,NaN,한식,제주시,33.488651,126.492164,호텔 레오 제주,33.489752,126.488260,0.382,NaN,0,4102,"맛, 맛있어요, 갈치, 맛집, 방문, 먹고, 맛있게, 곳, 맛있고, 조림, 반찬, ..."
1674,올레삼다정,"농림축산식품부 제공,안심식당",한식,서귀포시,33.253101,126.569837,호텔 케니 서귀포,33.250226,126.565131,0.542,4.65,2314,322,"맛, 갈치, 방문, 먹고, 맛있게, 곳, 조림, 반찬, 밥, 회, 구이, 메뉴, 게..."


In [ ]:
# final_df.to_csv('/content/drive/My Drive/jeju_review_data_final.csv', encoding='utf-8-sig', index=False)

In [ ]:
final_df['숙박업명'].value_counts(dropna=False) # 호텔별로 추천하는 식당 데이터 갯수

숙박업명
골든데이지호텔서귀포오션              75
제주 노블피아 호텔                74
오션팰리스                     73
헤이, 서귀포                   72
호텔 케니 서귀포                 71
신신호텔 서귀포점                 70
카사로마 호텔                   66
제주 센트럴 시티                 59
신신호텔 천지연                  58
신라스테이제주                   57
아시아호텔 제주                  57
호텔위드제주                    56
위드시티호텔                    56
호텔 위드스테이 제주               56
호텔 앨리스 앤 트렁크              55
노형호텔                      55
어반스테이 제주연동                52
호텔더원                      52
호텔 레오 제주                  52
롯데호텔 제주                   51
베스트웨스턴 제주호텔               51
롯데시티호텔 제주                 51
엠버호텔                      51
엠버시티호텔(구.아이비호텔)           49
그랜드하얏트 제주                 49
오션 스위츠 제주 호텔              49
호텔 스카이파크 제주 1호점           48
글로스터 제주                   47
신신호텔 제주공항                 46
제주 리젠트마린 더 블루             46
휘슬락                       45
어반아일랜드호텔                  45
베니키아 중문 호텔                42
아이진호텔                     41
호텔 시리우스  

In [ ]:
final_df.head()

,식당명,식당정보,대분류명,도시명,식당위도,식당경도,숙박업명,숙박위도,숙박경도,숙박_식당 거리(km),평점,방문자/블로그리뷰,검색량합계값,리뷰_키워드
0,60빈스,NaN,한식,서귀포시,33.241917,126.536242,돔베리조트,33.241433,126.531083,0.483,4.34,1223,875,"[맛, 맛있어요, 맛집, 곳, 맛있고, 뷰, 집, 가격, 바다, 분위기, 커피, 카..."
1,88버거,NaN,피자.치킨.햄버거,서귀포시,33.249246,126.567120,골든데이지호텔서귀포오션,33.249226,126.567572,0.042,4.71,13409,2984,"[맛, 맛있어요, 곳, 집, 푸, 말, 세트, 짐, 역시, 재료, 맛있어, 매콤, ..."
2,88버거,NaN,피자.치킨.햄버거,서귀포시,33.249246,126.567120,신신호텔 서귀포점,33.249515,126.557616,0.884,4.71,13409,2984,"[맛, 맛있어요, 곳, 집, 푸, 말, 세트, 짐, 역시, 재료, 맛있어, 매콤, ..."
3,88버거,NaN,피자.치킨.햄버거,서귀포시,33.249246,126.567120,신신호텔 천지연,33.249515,126.557616,0.884,4.71,13409,2984,"[맛, 맛있어요, 곳, 집, 푸, 말, 세트, 짐, 역시, 재료, 맛있어, 매콤, ..."
4,88버거,NaN,피자.치킨.햄버거,서귀포시,33.249246,126.567120,오션팰리스,33.246044,126.562304,0.572,4.71,13409,2984,"[맛, 맛있어요, 곳, 집, 푸, 말, 세트, 짐, 역시, 재료, 맛있어, 매콤, ..."


# TF-IDF 벡터라이징

In [ ]:
final_df = pd.read_csv('/content/drive/MyDrive/jeju_review_data_final.csv')

In [ ]:
final_df.sample(3)

,식당명,식당정보,대분류명,도시명,식당위도,식당경도,숙박업명,숙박위도,숙박경도,숙박_식당 거리(km),평점,방문자/블로그리뷰,검색량합계값,리뷰_키워드
2663,하영횟집,"농림축산식품부 제공,안심식당",일식.회,서귀포시,33.245268,126.522259,제주 골드원 호텔 스위트,33.244196,126.521955,0.123,4.40,419,285,"['맛', '맛있어요', '맛집', '방문', '먹고', '맛있게', '곳', '맛..."
455,돈물국수,NaN,한식,제주시,33.514017,126.535400,휘슬락,33.517351,126.527800,0.796,4.44,499,1883,"['맛', '맛있어요', '맛집', '방문', '먹고', '맛있게', '곳', '반..."
349,대도식당,제주 여행 길에 만나 서귀포 맛집 김치복국,한식,서귀포시,33.243991,126.562485,골든데이지호텔서귀포오션,33.249226,126.567572,0.750,4.27,817,972,"['맛', '맛있어요', '맛집', '방문', '먹고', '맛있게', '곳', '반..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
import re

def clean_keyword_list(keyword_str):
    # 대괄호 제거
    clean_str = re.sub(r'[\[\]]', '', keyword_str)
    # 작은 따옴표 제거
    clean_str = clean_str.replace("'", "")
    # 단어들 사이에 쉼표 추가
    clean_str = ', '.join(clean_str.split(', '))
    return clean_str

# '리뷰_키워드' 열에 함수 적용
final_df['리뷰_키워드'] = final_df['리뷰_키워드'].apply(clean_keyword_list)

In [ ]:
final_df['리뷰_키워드'].sample(3)

1141    맛, 맛있어요, 맛집, 방문, 먹고, 맛있게, 곳, 밥, 맛있었어요, 회, 구이, ...
1573    맛, 맛있어요, 고기, 맛집, 방문, 먹고, 맛있게, 곳, 맛있고, 반찬, 맛있었어...
1532    맛, 맛있어요, 갈치, 맛집, 먹고, 맛있게, 조림, 밥, 맛있었어요, 회, 구이,...
Name: 리뷰_키워드, dtype: object

In [ ]:
# 1180개 식당(중복포함)과, 488개 키워드
tfidf_matrix = tfidf.fit_transform(final_df['리뷰_키워드'])
tfidf_matrix.shape

(2898, 520)

In [ ]:
#코사인 유사도
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.01465192, 0.01465192, ..., 0.05241975, 0.05241975,
        0.05241975],
       [0.01465192, 1.        , 1.        , ..., 0.01595523, 0.01595523,
        0.01595523],
       [0.01465192, 1.        , 1.        , ..., 0.01595523, 0.01595523,
        0.01595523],
       ...,
       [0.05241975, 0.01595523, 0.01595523, ..., 1.        , 1.        ,
        1.        ],
       [0.05241975, 0.01595523, 0.01595523, ..., 1.        , 1.        ,
        1.        ],
       [0.05241975, 0.01595523, 0.01595523, ..., 1.        , 1.        ,
        1.        ]])

In [ ]:
cosine_sim.shape

(2898, 2898)

In [ ]:
# 각 식당별 인덱스(고유번호) 부여
indices = pd.Series(final_df.index, index=final_df['식당명']).drop_duplicates()

## 추천시스템

In [ ]:
# !pip install konlpy
import folium
from sklearn.metrics.pairwise import linear_kernel
import os
from google.colab import files
from konlpy.tag import Okt

# TF-IDF 벡터 생성 함수
def get_user_input_vector(user_input, tfidf_model):
    return tfidf_model.transform([user_input])


def get_recommendations_by_user_input_with_hotel(user_input, hotel_name, tfidf_model, cosine_sim=cosine_sim):
    # 호텔에 부합하는 행들 필터링
    hotel_indices = final_df[final_df['숙박업명'] == hotel_name].index

    # TF-IDF 벡터 생성
    user_tfidf_vector = get_user_input_vector(user_input, tfidf_model)

    # 사용자 입력과 호텔 필터링을 고려한 코사인 유사도 계산
    cosine_sim_user = linear_kernel(user_tfidf_vector, tfidf_matrix[hotel_indices])

    # 유사도가 높은 순으로 정렬
    sim_scores = list(enumerate(cosine_sim_user[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 상위 5개 식당 추출
    sim_scores = sim_scores[:5]
    restaurant_indices = [hotel_indices[i[0]] for i in sim_scores]

    # 추천 식당과 유사도 반환
    recommended_restaurants = final_df.iloc[restaurant_indices]
    similarity_scores = [round(i[1], 3) for i in sim_scores]

    return recommended_restaurants, similarity_scores

def recommend_restaurant():
    user_hotel = input("어느 호텔에서 묵고 계신가요? ")

    # 입력한 호텔명이 데이터에 있는지 확인
    if user_hotel not in final_df['숙박업명'].values:
        print("입력하신 호텔은 존재하지 않습니다.")
        return

    # user_food_type = input("어떤 음식을 찾으시나요? ('한식', '피자.치킨.햄버거', '일식.회', '중식', '베이커리.디저트', '카페', '양식', '고기', '분식', '주점및주류판매', '기타': ")

    # # 입력한 음식 종류에 해당하는 데이터 필터링
    # filtered_df = final_df[final_df['대분류명'] == user_food_type]

    # if filtered_df.empty:
    #     print("입력하신 음식 종류의 식당이 존재하지 않습니다.")
    #     return

    user_input = input("어떤 식당을 찾으시나요? ")

    # 호텔과 사용자 입력에 기반한 식당 추천 및 유사도 가져오기
    recommended_restaurants, similarity_scores = get_recommendations_by_user_input_with_hotel(user_input, user_hotel, tfidf, cosine_sim)

    if recommended_restaurants.empty:
        print("입력하신 조건에 부합하는 식당이 없습니다.")
    else:
        print("입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:")
        for restaurant, score in zip(recommended_restaurants['식당명'], similarity_scores):
            print(f"식당명: {restaurant} / 유사도: {score}")

        # 지도의 중심을 사용자가 묵고 있는 호텔의 위치로 설정
        hotel_location = final_df[final_df['숙박업명'] == user_hotel][['숙박위도', '숙박경도']].values[0]
        map_center = [hotel_location[0], hotel_location[1]]
        map_ = folium.Map(location=map_center, zoom_start=15)

        # 호텔 위치 마커 추가
        popup_html_hotel = f"""
        <div style="width: 300px;">
            <b style="font-size: 16px;">호텔명: {user_hotel}</b><br>
            <span style="font-size: 14px;">위치: {hotel_location[0]}, {hotel_location[1]}</span><br>
        </div>
        """

        folium.Marker(
            location=map_center,
            popup=folium.Popup(popup_html_hotel, max_width=300),
            icon=folium.Icon(color='red')
        ).add_to(map_)

        # 추천 식당 위치 마커 추가
        for idx, (idx_df, row) in enumerate(recommended_restaurants.iterrows()):
            # HTML 팝업 내용을 구성
            popup_html = f"""
            <div style="width: 300px;">
                <b style="font-size: 16px;">식당명: {row['식당명']}</b><br>
                <span style="font-size: 14px;">평점: {row['평점']}<br>
                방문자/블로그리뷰: {(row['방문자/블로그리뷰'])} 건<br>
                숙박_식당 거리: {(row['숙박_식당 거리(km)'])} km</span><br>

            """

            folium.Marker(
                location=[row['식당위도'], row['식당경도']],
                popup=popup_html,
                icon=folium.Icon(color='blue')
            ).add_to(map_)

        # 지도를 HTML 파일로 저장
        map_.save('recommended_restaurants_map.html')
        print("추천된 식당의 위치가 지도에 표시되었습니다. 'recommended_restaurants_map.html' 파일을 확인하세요.")

        # 현재 작업 디렉토리 확인
        cwd = os.getcwd()
        print(f"현재 작업 디렉토리: {cwd}")

        # Colab에서 파일 다운로드 링크 제공
        from google.colab import files
        files.download('recommended_restaurants_map.html')



recommend_restaurant()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 51.6 MB/s eta 0:00:00
어느 호텔에서 묵고 계신가요? 신라스테이제주
어떤 식당을 찾으시나요? 고등어 회 맛있고 반찬 다양하고 서비스 주는 맛집
입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:
식당명: 제주바당 / 유사도: 0.19
식당명: 고사리식당 / 유사도: 0.153
식당명: 황금어장 / 유사도: 0.138
식당명: 유리네식당 / 유사도: 0.136
식당명: 엉덩물 / 유사도: 0.09
추천된 식당의 위치가 지도에 표시되었습니다. 'recommended_restaurants_map.html' 파일을 확인하세요.
현재 작업 디렉토리: /content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

현재 사용 중인 TF-IDF와 코사인 유사도 기반의 추천 시스템은 단어 단위로 유사도를 계산하기 때문에, 사용자가 입력한 문장이나 단어의 조합을 적절히 인식하지 못할 수 있습니다

In [ ]:
final_df[final_df['식당명'] == '서귀포고기국수']['리뷰_키워드'].unique()

array(['맛, 맛있어요, 고기, 맛집, 먹고, 곳, 맛있고, 반찬, 집, 전복, 국물, 게, 국수, 김치, 짱, 맛있어, 간, 고기국수, 국, 서귀포, 무, 면, 반, 맛있다고, 물, 소, 죽, 맛있, 김, 찬, 만두, 복, 맛있다, 좋음, 찾아'],
      dtype=object)

In [ ]:
# def get_user_input_vector(user_input, tfidf_model):
#     return tfidf_model.transform([user_input])


# def get_recommendations_by_user_input_with_hotel(user_input, hotel_name, tfidf_model, cosine_sim=cosine_sim):
#     # 호텔에 부합하는 행들 필터링
#     hotel_indices = final_df[final_df['숙박업명'] == hotel_name].index

#     # TF-IDF 벡터 생성
#     user_tfidf_vector = get_user_input_vector(user_input, tfidf_model)

#     # 사용자 입력과 호텔 필터링을 고려한 코사인 유사도 계산
#     cosine_sim_user = linear_kernel(user_tfidf_vector, tfidf_matrix[hotel_indices])

#     # 유사도가 높은 순으로 정렬
#     sim_scores = list(enumerate(cosine_sim_user[0]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # 상위 5개 식당 추출
#     sim_scores = sim_scores[:5]
#     restaurant_indices = [hotel_indices[i[0]] for i in sim_scores]

#     # 추천 식당과 유사도 반환
#     recommended_restaurants = final_df.iloc[restaurant_indices]['식당명']
#     similarity_scores = [round(i[1], 3) for i in sim_scores]

#     return recommended_restaurants, similarity_scores

# # 사용자에게 식당 추천하는 함수
# def recommend_restaurant():
#     user_hotel = input("어느 호텔에서 묵고 계신가요? ")

#     # 입력한 호텔명이 데이터에 있는지 확인
#     if user_hotel not in final_df['숙박업명'].values:
#         print("입력하신 호텔은 존재하지 않습니다.")
#         return

#     user_input = input("어떤 식당을 찾으시나요? ")

#     # 호텔과 사용자 입력에 기반한 식당 추천 및 유사도 가져오기
#     recommended_restaurants, similarity_scores = get_recommendations_by_user_input_with_hotel(user_input, user_hotel, tfidf, cosine_sim)

#     if recommended_restaurants.empty:
#         print("입력하신 조건에 부합하는 식당이 없습니다.")
#     else:
#         print("입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:")
#         for restaurant, score in zip(recommended_restaurants, similarity_scores):
#             print(f"식당명: {restaurant} / 유사도: {score}")


# recommend_restaurant()


어느 호텔에서 묵고 계신가요? 제주 노블피아 호텔
어떤 식당을 찾으시나요? 회 맛집 반찬 
입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:
식당명: 호림식당 / 유사도: 0.199
식당명: 벌집식당 / 유사도: 0.182
식당명: 현재식당 / 유사도: 0.178
식당명: 대도식당 / 유사도: 0.176
식당명: 서귀포고기국수 / 유사도: 0.172


In [ ]:
# import folium
# from sklearn.metrics.pairwise import linear_kernel

# def get_user_input_vector(user_input, tfidf_model):
#     return tfidf_model.transform([user_input])

# def get_recommendations_by_user_input_with_hotel(user_input, hotel_name, tfidf_model, cosine_sim=cosine_sim):
#     # 호텔에 부합하는 행들 필터링
#     hotel_indices = final_df[final_df['숙박업명'] == hotel_name].index

#     # TF-IDF 벡터 생성
#     user_tfidf_vector = get_user_input_vector(user_input, tfidf_model)

#     # 사용자 입력과 호텔 필터링을 고려한 코사인 유사도 계산
#     cosine_sim_user = linear_kernel(user_tfidf_vector, tfidf_matrix[hotel_indices])

#     # 유사도가 높은 순으로 정렬
#     sim_scores = list(enumerate(cosine_sim_user[0]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # 상위 5개 식당 추출
#     sim_scores = sim_scores[:5]
#     restaurant_indices = [hotel_indices[i[0]] for i in sim_scores]

#     # 추천 식당과 유사도 반환
#     recommended_restaurants = final_df.iloc[restaurant_indices]
#     similarity_scores = [round(i[1], 3) for i in sim_scores]

#     return recommended_restaurants, similarity_scores

# def recommend_restaurant():
#     user_hotel = input("어느 호텔에서 묵고 계신가요? ")

#     # 입력한 호텔명이 데이터에 있는지 확인
#     if user_hotel not in final_df['숙박업명'].values:
#         print("입력하신 호텔은 존재하지 않습니다.")
#         return

#     user_input = input("어떤 식당을 찾으시나요? ")

#     # 호텔과 사용자 입력에 기반한 식당 추천 및 유사도 가져오기
#     recommended_restaurants, similarity_scores = get_recommendations_by_user_input_with_hotel(user_input, user_hotel, tfidf, cosine_sim)

#     if recommended_restaurants.empty:
#         print("입력하신 조건에 부합하는 식당이 없습니다.")
#     else:
#         print("입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:")
#         for restaurant, score in zip(recommended_restaurants['식당명'], similarity_scores):
#             print(f"식당명: {restaurant} / 유사도: {score}")

#         # 지도의 중심을 사용자가 묵고 있는 호텔의 위치로 설정
#         hotel_location = final_df[final_df['숙박업명'] == user_hotel][['숙박위도', '숙박경도']].values[0]
#         map_center = [hotel_location[0], hotel_location[1]]
#         map_ = folium.Map(location=map_center, zoom_start=15)

#         # 호텔 위치 마커 추가
#         folium.Marker(
#             location=map_center,
#             popup=f"{user_hotel} (호텔)",
#             icon=folium.Icon(color='red')
#         ).add_to(map_)

#         # 추천 식당 위치 마커 추가
#         for idx, (idx_df, row) in enumerate(recommended_restaurants.iterrows()):
#             folium.Marker(
#                 location=[row['식당위도'], row['식당경도']],
#                 popup=f"{row['식당명']} (유사도: {similarity_scores[idx]})",
#                 icon=folium.Icon(color='blue')
#             ).add_to(map_)

#         # 지도를 HTML 파일로 저장
#         map_.save('recommended_restaurants_map.html')
#         print("추천된 식당의 위치가 지도에 표시되었습니다. 'recommended_restaurants_map.html' 파일을 확인하세요.")

# # 제주도 식당 총 갯수 출력
# print(f'제주도 식당 총 갯수: {len(final_df["식당명"].unique())}개')

# recommend_restaurant()


제주도 식당 총 갯수: 692개
어느 호텔에서 묵고 계신가요? 신라스테이제주
어떤 식당을 찾으시나요? 고기국수 맛집
입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:
식당명: 삼무국수 / 유사도: 0.252
식당명: 제주부엌 / 유사도: 0.247
식당명: 국수만찬 / 유사도: 0.241
식당명: 삼대국수회관신제주점 / 유사도: 0.24
식당명: 올래국수 / 유사도: 0.208
추천된 식당의 위치가 지도에 표시되었습니다. 'recommended_restaurants_map.html' 파일을 확인하세요.


In [ ]:
# import folium
# from sklearn.metrics.pairwise import linear_kernel
# import os
# import webbrowser

# def get_user_input_vector(user_input, tfidf_model):
#     return tfidf_model.transform([user_input])

# def get_recommendations_by_user_input_with_hotel(user_input, hotel_name, tfidf_model, cosine_sim=cosine_sim):
#     # 호텔에 부합하는 행들 필터링
#     hotel_indices = final_df[final_df['숙박업명'] == hotel_name].index

#     # TF-IDF 벡터 생성
#     user_tfidf_vector = get_user_input_vector(user_input, tfidf_model)

#     # 사용자 입력과 호텔 필터링을 고려한 코사인 유사도 계산
#     cosine_sim_user = linear_kernel(user_tfidf_vector, tfidf_matrix[hotel_indices])

#     # 유사도가 높은 순으로 정렬
#     sim_scores = list(enumerate(cosine_sim_user[0]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # 상위 5개 식당 추출
#     sim_scores = sim_scores[:5]
#     restaurant_indices = [hotel_indices[i[0]] for i in sim_scores]

#     # 추천 식당과 유사도 반환
#     recommended_restaurants = final_df.iloc[restaurant_indices]
#     similarity_scores = [round(i[1], 3) for i in sim_scores]

#     return recommended_restaurants, similarity_scores

# def recommend_restaurant():
#     user_hotel = input("어느 호텔에서 묵고 계신가요? ")

#     # 입력한 호텔명이 데이터에 있는지 확인
#     if user_hotel not in final_df['숙박업명'].values:
#         print("입력하신 호텔은 존재하지 않습니다.")
#         return

#     user_input = input("어떤 식당을 찾으시나요? ")

#     # 호텔과 사용자 입력에 기반한 식당 추천 및 유사도 가져오기
#     recommended_restaurants, similarity_scores = get_recommendations_by_user_input_with_hotel(user_input, user_hotel, tfidf, cosine_sim)

#     if recommended_restaurants.empty:
#         print("입력하신 조건에 부합하는 식당이 없습니다.")
#     else:
#         print("입력하신 조건과 호텔에 부합하는 식당을 아래와 같이 추천드립니다:")
#         for restaurant, score in zip(recommended_restaurants['식당명'], similarity_scores):
#             print(f"식당명: {restaurant} / 유사도: {score}")

#         # 지도의 중심을 사용자가 묵고 있는 호텔의 위치로 설정
#         hotel_location = final_df[final_df['숙박업명'] == user_hotel][['숙박위도', '숙박경도']].values[0]
#         map_center = [hotel_location[0], hotel_location[1]]
#         map_ = folium.Map(location=map_center, zoom_start=15)

#         # 호텔 위치 마커 추가
#         folium.Marker(
#             location=map_center,
#             popup=f"{user_hotel} (호텔)",
#             icon=folium.Icon(color='red')
#         ).add_to(map_)

#         # 추천 식당 위치 마커 추가
#         for idx, (idx_df, row) in enumerate(recommended_restaurants.iterrows()):
#             folium.Marker(
#                 location=[row['식당위도'], row['식당경도']],
#                 popup=f"{row['식당명']} (유사도: {similarity_scores[idx]})",
#                 icon=folium.Icon(color='blue')
#             ).add_to(map_)

#         # 지도를 HTML 파일로 저장
#         map_.save('recommended_restaurants_map.html')
#         print("추천된 식당의 위치가 지도에 표시되었습니다. 'recommended_restaurants_map.html' 파일을 확인하세요.")

#         # 현재 작업 디렉토리 확인
#         cwd = os.getcwd()
#         print(f"현재 작업 디렉토리: {cwd}")

#         # HTML 파일 경로
#         file_path = os.path.join(cwd, 'recommended_restaurants_map.html')

#         # 웹 브라우저에서 파일 열기
#         webbrowser.open(f'file://{file_path}')

# # 제주도 식당 총 갯수 출력
# print(f'제주도 식당 총 갯수: {len(final_df["식당명"].unique())}개')

# recommend_restaurant()
